In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import ast # para la transformacion de str a array
#import requests
#import matplotlib.pyplot as plt
from PIL import Image
import re
import pandasql as sql # para trabajar con los datasets
from tqdm import tqdm # barra de progreso
#import tensorflow as tf
from concurrent.futures import ThreadPoolExecutor # para paralelismo
from concurrent.futures import as_completed  # para paralelismo

import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration

import spacy # para metrica de similitud

In [72]:
%pip install pandasql

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 1.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.4/602.4 kB 2.2 MB/s eta 0:00:00-:--:--
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26816 sha256=038c01fbc0bfde7fb7a5a26d65caec5e639018bb7271898f7e52216e94140014
  Stored in directory: /home/angel/.var/app/com.visualstudio.code/cache/pip/wheels/68/5d/a5/edc271b998f909801d7956959f699b976cc9896075dc47c153
Successfully built pandasql
Note: you may need to restart the kernel to use updated packages.


In [23]:
from concurrent.futures import ThreadPoolExecutor # para paralelismo
from concurrent.futures import as_completed  # para paralelismo
import pandas as pd
from tqdm import tqdm
import pandasql as sql
from PIL import Image

from semantic_text_similarity.models import WebBertSimilarity
from semantic_text_similarity.models import ClinicalBertSimilarity

In [ ]:
%pip install semantic-text-similarity

  Using cached semantic_text_similarity-1.0.3-py3-none-any.whl.metadata (2.6 kB)
  Using cached strsim-0.0.3-py3-none-any.whl.metadata (19 kB)
  Using cached fuzzywuzzy-0.18.0-py2.py3-none-any.whl.metadata (4.9 kB)
  Using cached pytorch_transformers-1.1.0-py3-none-any.whl.metadata (21 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
Using cached semantic_text_similarity-1.0.3-py3-none-any.whl (416 kB)
Using cached pytorch_transformers-1.1.0-py3-none-any.whl (158 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 2.1 MB/s eta 0:00:00m eta 0:00:010:00:01
Using cached strsim-0.0.3-py3-none-any.whl (42 kB)
Using cached fuzzywuzzy-0.18.0-py2.py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 2.2 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 2.2 MB/s eta 0:00:000:00:01m eta 0:00:01
Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 

In [2]:
#Metrica semantica texto 
from semantic_text_similarity.models import WebBertSimilarity #(1)

/home/angel/miniconda3/envs/TFM_2/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [3]:
import gc

gc.collect()

import torch
torch.cuda.empty_cache()


torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |\n|---------------------------------------------------------------------------|\n| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from large pool |      0 B   |      0 B   |

In [16]:
%pip install -U bitsandbytes

Note: you may need to restart the kernel to use updated packages.


In [4]:

from transformers import BitsAndBytesConfig

model_id = "llava-hf/llava-1.5-7b-hf"; #llava-hf/llava-1.5-7b-hf modelos de mas parametros. llava-1.5-13b-hf  llava-hf/llama3-llava-next-8b-hf llava-hf/bakLlava-v1-hf

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

model= LlavaForConditionalGeneration.from_pretrained(
    model_id, 
    torch_dtype=torch.float16, 
    low_cpu_mem_usage=True,
    #load_in_4bit=True
    quantization_config=bnb_config
).to(0);

processor = AutoProcessor.from_pretrained(model_id);

Loading checkpoint shards:  67%|██████▋   | 2/3 [00:10<00:05,  5.18s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacity of 3.70 GiB of which 28.88 MiB is free. Including non-PyTorch memory, this process has 3.21 GiB memory in use. Of the allocated memory 3.03 GiB is allocated by PyTorch, and 98.13 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [5]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

# Carga de dataset procesados

In [6]:
dataset_humano_cifar=unpickle('dataset_humano_cifar_250_v4_29_06_2024.pickle')
dataset_animal_cifar=unpickle('dataset_animal_cifar_250_v3_29_06_2024.pickle')
dataset_objeto_cifar=unpickle('dataset_objeto_cifar_250_v3_29_06_2024.pickle')
dataset_paisaje_cifar=unpickle('dataset_paisaje_cifar_250_v3_29_06_2024.pickle')

In [5]:
dataset_humano_cifar

,b'filenames',b'batch_label',b'fine_labels',b'coarse_labels',b'data',etiqueta,singularidad,categoria
58,b'black_woman_s_001310.png',None,98,14,"[165, 164, 139, 122, 113, 93, 74, 63, 67, 70, ...",woman,singular,Human
65,b'female_child_s_000292.png',None,35,14,"[220, 208, 213, 232, 239, 230, 163, 143, 150, ...",girl,singular,Human
77,b'man_s_000841.png',None,46,14,"[100, 117, 81, 92, 110, 152, 129, 107, 125, 11...",man,singular,Human
90,b'man_s_000737.png',None,46,14,"[97, 93, 94, 94, 88, 91, 91, 72, 57, 69, 71, 6...",man,singular,Human
92,b'girl_s_001964.png',None,35,14,"[61, 61, 62, 63, 63, 61, 60, 67, 81, 103, 128,...",girl,singular,Human
...,...,...,...,...,...,...,...,...
6149,b'abel_s_000450.png',None,46,14,"[213, 211, 212, 212, 212, 212, 212, 212, 212, ...",man,singular,Human
6159,b'woman_s_000104.png',None,98,14,"[121, 108, 95, 62, 47, 70, 55, 55, 95, 55, 33,...",woman,singular,Human
6216,b'young_girl_s_000399.png',None,35,14,"[229, 141, 123, 116, 146, 111, 93, 113, 42, 25...",girl,singular,Human
6223,b'infant_s_001111.png',None,2,14,"[85, 77, 75, 77, 71, 64, 99, 114, 95, 133, 158...",baby,singular,Human


In [5]:
dataset_animal_cifar

,b'filenames',b'batch_label',b'fine_labels',b'coarse_labels',b'data',etiqueta,singularidad,categoria
0,b'bos_taurus_s_000507.png',116,19,11,"[255, 255, 255, 255, 255, 255, 255, 255, 255, ...",cattle,plural,Animal
1,b'stegosaurus_s_000125.png',114,29,15,"[255, 253, 253, 253, 253, 253, 253, 253, 253, ...",dinosaur,singular,Animal
4,b'cichlid_s_000031.png',110,1,1,"[43, 32, 87, 127, 55, 48, 51, 87, 121, 113, 13...",aquarium_fish,singular,Animal
9,b'rogue_elephant_s_000421.png',98,31,11,"[255, 255, 255, 255, 255, 255, 255, 255, 255, ...",elephant,singular,Animal
17,b'wolf_pup_s_001323.png',111,97,8,"[167, 161, 170, 168, 163, 166, 165, 166, 167, ...",wolf,singular,Animal
...,...,...,...,...,...,...,...,...
904,b'winter_flounder_s_000509.png',None,32,1,"[234, 232, 232, 231, 230, 230, 230, 230, 230, ...",flatfish,singular,Animal
905,b'worm_snake_s_001265.png',None,78,15,"[60, 70, 71, 96, 137, 144, 63, 25, 8, 28, 116,...",snake,singular,Animal
906,b'stegosaurus_s_000069.png',None,29,15,"[71, 72, 80, 78, 76, 52, 35, 26, 16, 17, 16, 1...",dinosaur,singular,Animal
910,b'tiger_s_001191.png',None,88,8,"[241, 232, 244, 251, 250, 250, 248, 242, 245, ...",tiger,singular,Animal


In [6]:
dataset_objeto_cifar

,b'filenames',b'batch_label',b'fine_labels',b'coarse_labels',b'data',etiqueta,singularidad,categoria
2,b'mcintosh_s_000643.png',97,0,4,"[250, 248, 247, 248, 249, 249, 248, 248, 247, ...",apple,singular,Object
5,b'phone_s_002161.png',105,86,5,"[190, 191, 194, 191, 190, 192, 195, 197, 195, ...",telephone,singular,Object
7,b'beaker_s_000604.png',103,28,3,"[178, 175, 175, 175, 176, 176, 176, 176, 176, ...",cup,singular,Object
16,b'bike_s_000682.png',32,8,18,"[35, 27, 31, 77, 136, 168, 103, 101, 85, 63, 6...",bicycle,singular,Object
23,b'surveillance_system_s_000769.png',None,87,5,"[255, 252, 248, 243, 235, 224, 221, 222, 223, ...",television,singular,Object
...,...,...,...,...,...,...,...,...
1206,b'ballistic_missile_s_000772.png',None,69,19,"[199, 176, 169, 156, 141, 131, 124, 115, 111, ...",rocket,singular,Object
1209,b'mushroom_s_001277.png',None,51,4,"[160, 138, 134, 140, 133, 138, 140, 169, 157, ...",mushroom,singular,Object
1210,b'wing_chair_s_000220.png',None,20,6,"[246, 238, 227, 215, 211, 209, 210, 206, 188, ...",chair,singular,Object
1211,b'pickup_truck_s_000832.png',None,58,18,"[129, 124, 112, 115, 131, 133, 152, 165, 148, ...",pickup_truck,singular,Object


# Funcion de concatenacion

In [3]:
"Concatenacion preguntas"
def preguntar (pregunta):
    
    pregunta=f'USER: <image>\n"{pregunta}"\nASSISTANT:';
    
    return pregunta

# Preguntas

In [5]:
"PERSONAS"
sujeto_extraido='';

preguntas_max_humano=[
preguntar("what is the background of the image?"), 
preguntar("Is it rainy or sunny?"),
preguntar("Is it a closed or open place?"),
preguntar("Is the image bright?"),
preguntar("Are there elements in the image that can make visual noise?"),
preguntar("Are there a lot of different entities?"),
preguntar("Where is the image taken from?"),
preguntar("Are there humans?"),
preguntar("Are there animals?"),
preguntar(f"How many are {sujeto_extraido} there?"),
preguntar(f"Do {sujeto_extraido} have any elements that characterize them? Which ones?"),
preguntar(f"In what position are {sujeto_extraido} located?"),
preguntar("which actions are shown in the image"),
preguntar("Could you indicate the reasons why that action is performed?"),
preguntar(f"would the {sujeto_extraido} be able to perform any other action?"),
preguntar("Do you know what are the benefits or consequences of that action?"),
preguntar(f"How could you associate the {sujeto_extraido} with their environment to describe the action?"),
preguntar(f"Are {sujeto_extraido} young or old?"),
preguntar(f"Are There {sujeto_extraido} male?"),
preguntar(f"Are There {sujeto_extraido} female?"),                               
preguntar("What genders are there?"),
preguntar("What traits could be differentiated from other gender?"),
preguntar(f"What features could be differentiated from other types of {sujeto_extraido}?"), 
preguntar("What physical characteristics could be differentiated by other gender?"),
preguntar(f"Which {sujeto_extraido} is the male?"),
preguntar(f"{sujeto_extraido} have similar colors?"), # condicional plural el insert se encarga
preguntar(f"Are {sujeto_extraido} far away from each other?"), # condicional plural el insert se encarga
preguntar(f"Do {sujeto_extraido} interact with each other?"), # condicional plural el insert se encarga
preguntar(f"Have you been able to identify how many {sujeto_extraido} there are?"),
preguntar(f"If {sujeto_extraido} change positions could you have another prediction of the situation described?"),
preguntar("What is the most complicated characteristic or body part to define and determine?"),
preguntar(f"If {sujeto_extraido} were doing another action, would it be another type of {sujeto_extraido}?"),
preguntar(f"Are there elements in the image that could be noisy enough to intervene in the prediction that they are {sujeto_extraido}?"),
preguntar(f"What physical traits are considered for the prediction result to be {sujeto_extraido}?"),
preguntar(f"Why do you think identifying that they are {sujeto_extraido} in the image is easy?"),
preguntar(f"What makes you think the {sujeto_extraido} are actually {sujeto_extraido} and not something else?"),
preguntar(f"With which other {sujeto_extraido} would the prediction be most likely to give a better result?"),
preguntar(f"If there are subtle changes in skin color, clothing or posture, would the result of the prediction that they are {sujeto_extraido} be the same?"),
preguntar(f"Is there anything that might create doubt or uncertainty for you to say they are {sujeto_extraido}?"),
preguntar("Could you describe what is happening in the image?"),
preguntar("Could the situation you predicted be used in other areas?"),
preguntar("Could the situation you predicted have some example of counterfactual situation that could be useful?"),
preguntar("How could the situation you have predicted above be better?"),
preguntar("What would an image similar to the situation found in this image look like?"),
preguntar("Can you describe a similar counterexample to the situation you have predicted above?"),
preguntar(f"What is the task or purpose of {sujeto_extraido}?"),
preguntar("Could you in simpler words describe their function that you have predicted before?"),
preguntar("What could stand out from the situation described above and be useful in similar situations?"),
preguntar("Can you give essential concepts that are relevant to the situation described?"),
preguntar("What aspects of the image have been detected that may attract attention?"),
preguntar(f"Why has it been obtained that the prediction is that they are {sujeto_extraido}?"),
preguntar(f"How did you get the result that they are {sujeto_extraido}?"),
preguntar(f"What do you focus on to get the prediction that they are {sujeto_extraido}?"),
preguntar(f"What do you focus on {sujeto_extraido} to result in or say that they are {sujeto_extraido}?"),
preguntar(f"Do you focus on parts of the {sujeto_extraido} bodies to infer that they are {sujeto_extraido}?"),
preguntar(f"Do you focus on features or facial expressions of the {sujeto_extraido} to result in saying they are {sujeto_extraido}?"),
preguntar(f"Do you focus on the {sujeto_extraido} clothing to result in the {sujeto_extraido} being {sujeto_extraido}?"),
preguntar(f"Do you focus on the {sujeto_extraido} body shape to result in the result that they are {sujeto_extraido}?")
          ]    
len(preguntas_max_humano)

58

In [14]:
"ANIMALES"
preguntas_max_animales=[
preguntar("what is the background of the image?"),
preguntar("Is it rainy or sunny?"),
preguntar("Is it a closed or open place?"),
preguntar("Is the image bright?"),
preguntar("Are there elements in the image that can make visual noise?"),
preguntar("Are there a lot of different entities?"),
preguntar("Where is the image taken from?"),
preguntar("Are there humans?"),
preguntar("Are there animals?"),
preguntar(f"How many are {sujeto_extraido} there?"),
preguntar(f"Do {sujeto_extraido} have any elements that characterize them? Which ones?"),
preguntar(f"In what position are {sujeto_extraido} located?"),
preguntar("which actions are shown in the image"),
preguntar("Could you indicate the reasons why that action is performed?"),
preguntar(f"would the {sujeto_extraido} be able to perform any other action?"),
preguntar("Do you know what are the benefits or consequences of that action?"),
preguntar(f"How could you associate the {sujeto_extraido} with their environment to describe the action?"),
preguntar(f"Are {sujeto_extraido} young or old?"),
preguntar(f"Are There {sujeto_extraido} male?"),
preguntar(f"Are There {sujeto_extraido} famale?"),                      
preguntar("What genders are there?"),
preguntar(f"What features could be differentiated from other types of {sujeto_extraido}?"),
preguntar("What physical characteristics could be differentiated other gender?"),
preguntar(f"Which {sujeto_extraido} is the male?"),
preguntar(f"{sujeto_extraido} have similar colors?"), # condicionada plural 24
preguntar(f"Are {sujeto_extraido} far away from each other?"), # condicionada plural 25
preguntar(f"Do {sujeto_extraido} interact with each other?"), # condicionada plural 26
preguntar(f"Have you been able to identify how many {sujeto_extraido} there are?"),
preguntar(f"If the {sujeto_extraido} change positions could you have another prediction of the situation described?"),
preguntar("What is the most complicated characteristic or body part to define and determine?"),
preguntar(f"If the {sujeto_extraido} were doing another action, would it be another type of {sujeto_extraido}?"),
preguntar(f"Are there elements in the image that could be noisy enough to intervene in the prediction that they are {sujeto_extraido}?"),
preguntar(f"What physical traits are considered for the prediction result to be {sujeto_extraido}?"),
preguntar(f"Why do you think identifying that they are {sujeto_extraido} in the image is easy?"), ##---eficiencia
preguntar(f"What makes you think the {sujeto_extraido} are actually {sujeto_extraido} and not something else?"),
preguntar(f"With which other {sujeto_extraido} would the prediction be most likely to give a better result?"),
preguntar(f"If there are subtle changes in skin color or posture, would the result of the prediction that they are {sujeto_extraido} be the same?"),
preguntar(f"Is there anything that might create doubt or uncertainty for you to say they are {sujeto_extraido}?"),
preguntar("Could you describe what is happening in the image?"),
preguntar("Could the situation you predicted be used in other areas?"),
preguntar("Could the situation you predicted have some example of counterfactual situation that could be useful?"),
preguntar("How could the situation you have predicted above be better?"),
preguntar("What would an image similar to the situation found in this image look like?"),
preguntar("Can you describe a similar counterexample to the situation you have predicted above?"),
preguntar(f"What is the task or purpose of {sujeto_extraido}?"),
preguntar("Could you in simpler words describe their function that you have predicted before?"),
preguntar("What could stand out from the situation described above and be useful in similar situations?"),
preguntar("Can you give essential concepts that are relevant to the situation described?"),
preguntar("What aspects of the image have been detected that may attract attention?"),
preguntar(f"Why has it been obtained that the prediction is that they are {sujeto_extraido}?"),
preguntar(f"How did you get the result that they are {sujeto_extraido}?"),
preguntar(f"What do you focus on to get the prediction that they are {sujeto_extraido}?"),
preguntar(f"What do you focus on {sujeto_extraido} to result in or say that they are {sujeto_extraido}?"),
preguntar(f"Do you focus on parts of the {sujeto_extraido} bodies to infer that they are {sujeto_extraido}?"),
preguntar(f"Do you focus on features or facial expressions of the {sujeto_extraido} to result in saying they are {sujeto_extraido}?"),
preguntar(f"Do you focus on the {sujeto_extraido} skin color to result in the {sujeto_extraido} being {sujeto_extraido}?"),
preguntar(f"Do you focus on the {sujeto_extraido} body shape to result in the result that they are {sujeto_extraido}?")]

len(preguntas_max_animales)

57

In [15]:
"OBJETOS"
preguntas_max_objetos=[
preguntar("what is the background of the image?"), 
preguntar("Is it rainy or sunny?"),
preguntar("Is it a closed or open place?"),
preguntar("Is the image bright?"),
preguntar("Are there elements in the image that can make visual noise?"),
preguntar("Are there a lot of different entities?"),
preguntar("Where is the image taken from?"),
preguntar("Are there objects?"),
preguntar(f"How many are {sujeto_extraido} there?"),
preguntar(f"Do {sujeto_extraido} have any elements that characterize them? Which ones?"),
preguntar(f"In what position are {sujeto_extraido} located?"),
preguntar("which actions are shown in the image"),
preguntar("Could you indicate the reasons why that action is performed?"),
preguntar(f"would the {sujeto_extraido} be able to perform any other action?"),
preguntar("Do you know what are the benefits or consequences of that action?"),
preguntar(f"How could you associate the {sujeto_extraido} with their environment to describe the action?"),
preguntar(f"Are {sujeto_extraido} new or old?"),
preguntar(f"What features could be differentiated from other types of {sujeto_extraido}?"),
preguntar(f"What physical characteristics could be differentiated other {sujeto_extraido}?"),   
preguntar(f"{sujeto_extraido} have similar colors?"), # condicionada
preguntar(f"Are {sujeto_extraido} far away from each other?"), # condicionada
preguntar(f"Have you been able to identify how many {sujeto_extraido} there are?"),
preguntar(f"If the {sujeto_extraido} change positions could you have another prediction of the situation described?"),
preguntar("What is the most complicated characteristic to define and determine?"),
preguntar(f"If the {sujeto_extraido} were doing another action, would it be another type of {sujeto_extraido}?"),
preguntar(f"Are there elements in the image that could be noisy enough to intervene in the prediction that they are {sujeto_extraido}?"),
preguntar(f"What physical traits are considered for the prediction result to be {sujeto_extraido}?"),
preguntar(f"Why do you think identifying that they are {sujeto_extraido} in the image is easy?"),
preguntar(f"What makes you think the {sujeto_extraido} are actually {sujeto_extraido} and not something else?"),
preguntar(f"With which other {sujeto_extraido} would the prediction be most likely to give a better result?"),
preguntar(f"If there are subtle changes in color or posture, would the result of the prediction that they are {sujeto_extraido} be the same?"),
preguntar(f"Is there anything that might create doubt or uncertainty for you to say they are {sujeto_extraido}?"),
preguntar("Could you describe what is happening in the image?"),
preguntar("Could the situation you predicted be used in other areas?"),
preguntar("Could the situation you predicted have some example of counterfactual situation that could be useful?"),
preguntar("How could the situation you have predicted above be better?"),
preguntar("What would an image similar to the situation found in this image look like?"),
preguntar("Can you describe a similar counterexample to the situation you have predicted above?"),
preguntar(f"What is the task or purpose of {sujeto_extraido}?"),
preguntar("Could you in simpler words describe their function that you have predicted before?"),
preguntar("What could stand out from the situation described above and be useful in similar situations?"),
preguntar("Can you give essential concepts that are relevant to the situation described?"),
preguntar("What aspects of the image have been detected that may attract attention?"),
preguntar(f"Why has it been obtained that the prediction is that they are {sujeto_extraido}?"),
preguntar(f"How did you get the result that they are {sujeto_extraido}?"),
preguntar(f"What do you focus on to get the prediction that they are {sujeto_extraido}?"),
preguntar(f"What do you focus on {sujeto_extraido} to result in or say that they are {sujeto_extraido}?"),
preguntar(f"Do you focus on parts of the {sujeto_extraido} bodies to infer that they are {sujeto_extraido}?"),
preguntar(f"Do you focus on features of the {sujeto_extraido} to result in saying they are {sujeto_extraido}?"),
preguntar(f"Do you focus on the {sujeto_extraido} color to result in the {sujeto_extraido} being {sujeto_extraido}?"),
preguntar(f"Do you focus on the {sujeto_extraido} shape to result in the result that they are {sujeto_extraido}?")]

len(preguntas_max_objetos)

51

In [9]:
"PAISAJES"
def preguntas_max_paisajes_funcion(sujeto_extraido):
    preguntas_max_paisajes =[
    preguntar("what is the background of the image?"), 
    preguntar("Is it rainy or sunny?"),
    preguntar("Is it a closed or open place?"),
    preguntar("Is the image bright?"),
    preguntar("Are there elements in the image that can make visual noise?"),
    preguntar("Are there a lot of different entities?"),
    preguntar("Where is the image taken from?"),
    preguntar("Are there humans?"),
    preguntar("Are there animals?"),
    preguntar(f"How many are {sujeto_extraido} there?"),
    preguntar(f"Do {sujeto_extraido} have any elements that characterize them? Which ones?"),
    preguntar(f"In what position are {sujeto_extraido} located?"),
    preguntar("which actions are shown in the image"),
    preguntar("Could you indicate the reasons why that action is performed?"),
    preguntar(f"would the {sujeto_extraido} be able to perform any other action?"),
    preguntar("Do you know what are the benefits or consequences of that action?"),
    preguntar(f"How could you associate the {sujeto_extraido} with their environment to describe the action?"),
    preguntar(f"Are they {sujeto_extraido} young or old?"),
    preguntar(f"What features could be differentiated from other types of {sujeto_extraido}?"),
    preguntar(f"What physical characteristics could be differentiated from the opposite {sujeto_extraido}?"),
    preguntar(f"{sujeto_extraido} have similar colors?"), # coindicionada
    preguntar(f"Have you been able to identify how many {sujeto_extraido} there are?"),
    preguntar(f"If the {sujeto_extraido} change positions could you have another prediction of the situation described?"),
    preguntar(f"If the {sujeto_extraido} were doing another action, would it be another type of {sujeto_extraido}?"),
    preguntar(f"Are there elements in the image that could be noisy enough to intervene in the prediction that they are {sujeto_extraido}?"),
    preguntar(f"What physical traits are considered for the prediction result to be {sujeto_extraido}?"),
    preguntar(f"Why do you think identifying that they are {sujeto_extraido} in the image is easy?"),
    preguntar(f"What makes you think the {sujeto_extraido} are actually {sujeto_extraido} and not something else?"),
    preguntar(f"With which other kind of {sujeto_extraido} would the prediction be most likely to give a better result?"),
    preguntar(f"If there are subtle changes in color or position, would the result of the prediction that they are {sujeto_extraido} be the same?"),
    preguntar(f"Is there anything that might create doubt or uncertainty for you to say they are {sujeto_extraido}?"),
    preguntar("Could you describe what is happening in the image?"),
    preguntar("Could the situation you predicted be used in other areas?"),
    preguntar("Could the situation you predicted have some example of counterfactual situation that could be useful?"),
    preguntar("How could the situation you have predicted above be better?"),
    preguntar("What would an image similar to the situation found in this image look like?"),
    preguntar("Can you describe a similar counterfactual to the situation you have predicted above?"),
    preguntar(f"What is the task or purpose of {sujeto_extraido}?"),
    preguntar("Could you in simpler words describe their function that you have predicted before?"),
    preguntar("What could stand out from the situation described above and be useful in similar situations?"),
    preguntar("Can you give essential concepts that are relevant to the situation described?"),
    preguntar("What aspects of the image have been detected that may attract attention?"),
    preguntar(f"Why has it been obtained that the prediction is that they are {sujeto_extraido}?"),
    preguntar(f"How did you get the result that they are {sujeto_extraido}?"),
    preguntar(f"What do you focus on to get the prediction that they are {sujeto_extraido}?"),
    preguntar(f"What do you focus on {sujeto_extraido} to result in or say that they are {sujeto_extraido}?"),
    preguntar(f"Do you focus on parts of the {sujeto_extraido} bodies to infer that they are {sujeto_extraido}?"),
    preguntar(f"Do you focus on the {sujeto_extraido} body shape to result in the result that they are {sujeto_extraido}?")]

    return preguntas_max_paisajes

len(preguntas_max_paisajes)

48

# funcion para aplicar pregunta

In [17]:
"Necesario para sacar el sujeto de la descripcion:"
from transformers import pipeline
question_answerer = pipeline("question-answering", model='distilbert-base-cased-distilled-squad')


"Para sacar el sujeto de la respues de descripcion"
def sujeto(respuesta_pregunta_descripcion):
    
    "Para sacar sujeto de description:"
    # Colocar la variable dentro del contexto raw string
    contexto = r"""{}""".format(respuesta_pregunta_descripcion)


    result = question_answerer(question="Who is being talked about?", context=contexto)
    sujeto= result['answer']
    
    print(sujeto)
    
    return sujeto


"Para sacar la cantiadd del sujeto que se encuentra"
def cantidad(respuesta_pregunta_descripcion):
    "Para sacar la cantidad:"
    # coloca la variable dentro del contexto raw string
    contexto = r"""{}""".format(respuesta_pregunta_descripcion)


    result = question_answerer(question="What numerical quantity of main entity is being discussed?", context=contexto) # aqui poner preguntar por el numero en la frasde
    cantidad= result['answer']
    
    print(cantidad)
    
    return cantidad



"Respuestas"
def predit(grupo,indice): # grupo 1 humano, grupo 2 animal, grupo 3 objeto, grupo 4 paisaje
        


    if grupo ==1:
        preguntas_hacer=preguntas_max_humano
        dataset=dataset_humano_cifar
    elif grupo ==2:
        preguntas_hacer=preguntas_max_animales
        dataset=dataset_animal_cifar
    elif grupo==3:
        preguntas_hacer=preguntas_max_objetos
        dataset=dataset_objeto_cifar
    elif grupo==4:
        preguntas_hacer=preguntas_max_paisajes
        dataset=dataset_paisaje_cifar
    else:
        pass
        

    # Extraer la imagen en formato numpy
    image_numpy= dataset[b'data'][indice]
    
     #dimensionar correctamente
    image_numpy= image_numpy.reshape(3, 32, 32).transpose(1,2,0)
    
    # pasando  de numpy a PIL
    raw_image =Image.fromarray(np.uint8(image_numpy), 'RGB')
    
    
    #sujeto extraido
    sujeto_extraido=dataset["etiqueta"][indice]
    
    #categoria
    categoria = dataset["categoria"][indice]
    
    #singularidad
    singularidad= dataset["singularidad"][indice]
    
    """
    if singularidad =='plural':
        preguntas_hacer.insert(24,"{sujeto_extraido} have similar colors?")
        preguntas_hacer.insert(25,"Are {sujeto_extraido} far away from each other?")
        preguntas_hacer.insert(26,"Do {sujeto_extraido} interact with each other?")
    else:
        preguntas_hacer.pop(24)
        preguntas_hacer.pop(24)
        preguntas_hacer.pop(24)
    """
    
    
    # Mostrar la imagen JPEG
    #plt.imshow(image_numpy, interpolation='bilinear')
    #plt.axis('off')
    #plt.show()

    #para guardar las respuestas
    #preguntas=[];
    respuestas=[];
    
    
    
    for prompt in preguntas_hacer:
        # input
        inputs = processor(prompt, raw_image, return_tensors='pt', max_length=512, padding='longest', truncation=True).to(model.device, torch.float16)        
        # output
        #output = model.generate(**inputs, max_new_tokens=200, num_beams=4, early_stopping=True)
        output = model.generate(**inputs, max_new_tokens=250, do_sample=False)

        #auxiliar para extraer preguntas y respuestas
        entrada_texto = processor.decode(output[0][2:], skip_special_tokens=True)

        inicio_indice= entrada_texto.find("ER:") + 4;
        final_indice = entrada_texto.find("\nASSISTANT");
        pregunta= entrada_texto[inicio_indice:final_indice];

        inicio_indice = entrada_texto.find("\nASSISTANT:") + len("\nASSISTANT:");
        respuesta = entrada_texto[final_indice+12:];

        respuestas.append(respuesta)
    
    del inputs, output, raw_image#, preguntas ya las preguntas no sirven
    torch.cuda.empty_cache()
    return respuestas

In [18]:
# creacion de dataset de preguntas y respuestas
dataset_humano_respuestas=pd.DataFrame(columns=preguntas_max_humano)
dataset_animales_respuestas=pd.DataFrame(columns=preguntas_max_animales)
dataset_objetos_respuestas=pd.DataFrame(columns=preguntas_max_objetos)
dataset_paisajes_respuestas=pd.DataFrame(columns=preguntas_max_paisajes)

In [19]:
dataset_humano_cifar

,b'filenames',b'batch_label',b'fine_labels',b'coarse_labels',b'data',etiqueta,singularidad,categoria
58,b'black_woman_s_001310.png',None,98,14,"[165, 164, 139, 122, 113, 93, 74, 63, 67, 70, ...",woman,singular,Human
65,b'female_child_s_000292.png',None,35,14,"[220, 208, 213, 232, 239, 230, 163, 143, 150, ...",girl,singular,Human
77,b'man_s_000841.png',None,46,14,"[100, 117, 81, 92, 110, 152, 129, 107, 125, 11...",man,singular,Human
90,b'man_s_000737.png',None,46,14,"[97, 93, 94, 94, 88, 91, 91, 72, 57, 69, 71, 6...",man,singular,Human
92,b'girl_s_001964.png',None,35,14,"[61, 61, 62, 63, 63, 61, 60, 67, 81, 103, 128,...",girl,singular,Human
...,...,...,...,...,...,...,...,...
6149,b'abel_s_000450.png',None,46,14,"[213, 211, 212, 212, 212, 212, 212, 212, 212, ...",man,singular,Human
6159,b'woman_s_000104.png',None,98,14,"[121, 108, 95, 62, 47, 70, 55, 55, 95, 55, 33,...",woman,singular,Human
6216,b'young_girl_s_000399.png',None,35,14,"[229, 141, 123, 116, 146, 111, 93, 113, 42, 25...",girl,singular,Human
6223,b'infant_s_001111.png',None,2,14,"[85, 77, 75, 77, 71, 64, 99, 114, 95, 133, 158...",baby,singular,Human


In [6]:
indices_dataset_humano_cifar=dataset_humano_cifar.index
indices_dataset_animal_cifar=dataset_animal_cifar.index
indices_dataset_objeto_cifar=dataset_objeto_cifar.index
indices_dataset_paisaje_cifar=dataset_paisaje_cifar.index

In [4]:
try:
    for x in tqdm(indices_dataset_humano_cifar[:50]):
        respuestas=predit(1,x) # 1 porque es humano
        for columna, respuesta in zip(dataset_humano_respuestas.columns, respuestas):#dataset_humano_respuestas.columns:
                #for indice_respuestas in range(0,len(respuestas)):
            dataset_humano_respuestas.loc[x, columna] = respuesta
        del respuestas
except:
    print("error")

100%|██████████| 50/50 [4:03:08<00:00, 291.78s/it]  


In [13]:
dataset_humano_respuestas

,"USER: <image>\n""what is the background of the image?""\nASSISTANT:","USER: <image>\n""Is it rainy or sunny?""\nASSISTANT:","USER: <image>\n""Is it a closed or open place?""\nASSISTANT:","USER: <image>\n""Is the image bright?""\nASSISTANT:","USER: <image>\n""Are there elements in the image that can make visual noise?""\nASSISTANT:","USER: <image>\n""Are there a lot of different entities?""\nASSISTANT:","USER: <image>\n""Where is the image taken from?""\nASSISTANT:","USER: <image>\n""Are there humans?""\nASSISTANT:","USER: <image>\n""Are there animals?""\nASSISTANT:","USER: <image>\n""How many are there?""\nASSISTANT:",...,"USER: <image>\n""Can you give essential concepts that are relevant to the situation described?""\nASSISTANT:","USER: <image>\n""What aspects of the image have been detected that may attract attention?""\nASSISTANT:","USER: <image>\n""Why has it been obtained that the prediction is that they are ?""\nASSISTANT:","USER: <image>\n""How did you get the result that they are ?""\nASSISTANT:","USER: <image>\n""What do you focus on to get the prediction that they are ?""\nASSISTANT:","USER: <image>\n""What do you focus on to result in or say that they are ?""\nASSISTANT:","USER: <image>\n""Do you focus on parts of the bodies to infer that they are ?""\nASSISTANT:","USER: <image>\n""Do you focus on features or facial expressions of the to result in saying they are ?""\nASSISTANT:","USER: <image>\n""Do you focus on the clothing to result in the being ?""\nASSISTANT:","USER: <image>\n""Do you focus on the body shape to result in the result that they are ?""\nASSISTANT:"
2106,The background of the image is a blue and purp...,Sunny,It is an open place.,"Yes, the image is bright, featuring a woman wi...","Yes, there are elements in the image that can ...","No, the image only features one woman with lon...",The image is taken from a close-up perspective...,Yes,No,"There is only one person in the image, a woman...",...,"In the image, a woman with long blonde hair is...","In the image, a woman with long blonde hair is...","The image features a woman with long hair, and...",The result that the woman in the image is a pa...,"To predict that the image features a woman, I ...","In the image, the focus is on the woman's face...","Yes, in the image, the focus is on the woman's...","Yes, the image focuses on the facial features ...","Yes, the focus on the clothing in the image, s...","Yes, the focus on the body shape and the woman..."
2131,The background of the image is blue.,It is sunny in the image.,"It is an open place, as the girl is smiling an...","Yes, the image is bright, as it is a close-up ...","Yes, there are elements in the image that can ...","No, the image only shows a single person, a yo...",The image is taken from a close-up perspective...,Yes,No,There are two people in the image.,...,"In the image, a young woman is smiling and loo...","In the image, a young woman with blonde hair i...",The image shows a young woman with blonde hair...,The result that the person in the image is a y...,To predict that the person in the image is a w...,"In the image, the focus is on the young girl's...","Yes, the image shows a close-up of a woman's f...","In the image, the girl is smiling and has a br...","Yes, the image focuses on the girl's clothing,...","Yes, the image shows a young woman with a smil..."
2179,"The background of the image is blurry, which s...",It is sunny in the image.,"It is an open place, as the man is standing in...","Yes, the image is bright, as it is described a...","Yes, there are elements in the image that can ...","Yes, there are many different entities in the ...",The image is taken from the perspective of the...,"Yes, there are humans in the image.",No,"There are several people in the image, includi...",...,"In the image, a man is holding a camera and lo...","In the image, a man is holding a camera, which...","The image shows a man holding a camera, with a...",The result that the people in the image are 

In [1]:
dataset_humano_respuestas.to_pickle('dataset_humano_respuestas_parte1.pkl')

# 2 tanda dataset humano

In [2]:
try:
    for x in tqdm(indices_dataset_humano_cifar[50:100]):
        respuestas=predit(1,x) # 1 porque es humano
        for columna, respuesta in zip(dataset_humano_respuestas.columns, respuestas):#dataset_humano_respuestas.columns:
                #for indice_respuestas in range(0,len(respuestas)):
            dataset_humano_respuestas.loc[x, columna] = respuesta
        del respuestas
except:
    print("error")

100%|██████████| 50/50 [3:59:35<00:00, 287.51s/it]  


In [ ]:
dataset_humano_respuestas.to_pickle('dataset_humano_respuestas_parte2.pkl')

In [6]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [22]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:1024"

In [23]:
!#export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128
# Desactivar la paralelización en la biblioteca `tokenizers`
os.environ["TOKENIZERS_PARALLELISM"] = "true"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# 3ra tanda (parallismo)

In [28]:
for x in tqdm(indices_dataset_humano_cifar[100:150]):
    respuestas=predit(1,x) # 1 porque es humano
    for columna, respuesta in zip(dataset_humano_respuestas.columns, respuestas):#dataset_humano_respuestas.columns:
            #for indice_respuestas in range(0,len(respuestas)):
        dataset_humano_respuestas.loc[x, columna] = respuesta
    del respuestas


100%|██████████| 50/50 [4:02:43<00:00, 291.27s/it]  


In [29]:
dataset_humano_respuestas.to_pickle('dataset_humano_respuestas_parte3.pkl')

In [ ]:
# Tentativa

def respuestas(x):
    #try:
        respuestas=predit(1,x) # 1 porque es humano
        for columna, respuesta in zip(dataset_humano_respuestas.columns, respuestas):#dataset_humano_respuestas.columns:
                #for indice_respuestas in range(0,len(respuestas)):
            dataset_humano_respuestas.loc[x, columna] = respuesta
        del respuestas
    #except:
    #    print("error")
        
#for x in tqdm(indices_dataset_humano_cifar[:50]):
    
with ThreadPoolExecutor() as executor:
    for aux in tqdm(executor.map(respuestas,indices_dataset_humano_cifar[100:150])):
        pass

# 4 Tanda

In [30]:
for x in tqdm(indices_dataset_humano_cifar[150:200]):
    respuestas=predit(1,x) # 1 porque es humano
    for columna, respuesta in zip(dataset_humano_respuestas.columns, respuestas):#dataset_humano_respuestas.columns:
            #for indice_respuestas in range(0,len(respuestas)):
        dataset_humano_respuestas.loc[x, columna] = respuesta
    del respuestas

100%|██████████| 50/50 [4:02:51<00:00, 291.43s/it]  


In [32]:
dataset_humano_respuestas.to_pickle('dataset_humano_respuestas_parte4.pkl')

In [1]:
dataset_humano_respuestas

,"USER: <image>\n""what is the background of the image?""\nASSISTANT:","USER: <image>\n""Is it rainy or sunny?""\nASSISTANT:","USER: <image>\n""Is it a closed or open place?""\nASSISTANT:","USER: <image>\n""Is the image bright?""\nASSISTANT:","USER: <image>\n""Are there elements in the image that can make visual noise?""\nASSISTANT:","USER: <image>\n""Are there a lot of different entities?""\nASSISTANT:","USER: <image>\n""Where is the image taken from?""\nASSISTANT:","USER: <image>\n""Are there humans?""\nASSISTANT:","USER: <image>\n""Are there animals?""\nASSISTANT:","USER: <image>\n""How many are there?""\nASSISTANT:",...,"USER: <image>\n""Can you give essential concepts that are relevant to the situation described?""\nASSISTANT:","USER: <image>\n""What aspects of the image have been detected that may attract attention?""\nASSISTANT:","USER: <image>\n""Why has it been obtained that the prediction is that they are ?""\nASSISTANT:","USER: <image>\n""How did you get the result that they are ?""\nASSISTANT:","USER: <image>\n""What do you focus on to get the prediction that they are ?""\nASSISTANT:","USER: <image>\n""What do you focus on to result in or say that they are ?""\nASSISTANT:","USER: <image>\n""Do you focus on parts of the bodies to infer that they are ?""\nASSISTANT:","USER: <image>\n""Do you focus on features or facial expressions of the to result in saying they are ?""\nASSISTANT:","USER: <image>\n""Do you focus on the clothing to result in the being ?""\nASSISTANT:","USER: <image>\n""Do you focus on the body shape to result in the result that they are ?""\nASSISTANT:"
2106,The background of the image is a blue and purp...,Sunny,It is an open place.,"Yes, the image is bright, featuring a woman wi...","Yes, there are elements in the image that can ...","No, the image only features one woman with lon...",The image is taken from a close-up perspective...,Yes,No,"There is only one person in the image, a woman...",...,"In the image, a woman with long blonde hair is...","In the image, a woman with long blonde hair is...","The image features a woman with long hair, and...",The result that the woman in the image is a pa...,"To predict that the image features a woman, I ...","In the image, the focus is on the woman's face...","Yes, in the image, the focus is on the woman's...","Yes, the image focuses on the facial features ...","Yes, the focus on the clothing in the image, s...","Yes, the focus on the body shape and the woman..."
2131,The background of the image is blue.,It is sunny in the image.,"It is an open place, as the girl is smiling an...","Yes, the image is bright, as it is a close-up ...","Yes, there are elements in the image that can ...","No, the image only shows a single person, a yo...",The image is taken from a close-up perspective...,Yes,No,There are two people in the image.,...,"In the image, a young woman is smiling and loo...","In the image, a young woman with blonde hair i...",The image shows a young woman with blonde hair...,The result that the person in the image is a y...,To predict that the person in the image is a w...,"In the image, the focus is on the young girl's...","Yes, the image shows a close-up of a woman's f...","In the image, the girl is smiling and has a br...","Yes, the image focuses on the girl's clothing,...","Yes, the image shows a young woman with a smil..."
2179,"The background of the image is blurry, which s...",It is sunny in the image.,"It is an open place, as the man is standing in...","Yes, the image is bright, as it is described a...","Yes, there are elements in the image that can ...","Yes, there are many different entities in the ...",The image is taken from the perspective of the...,"Yes, there are humans in the image.",No,"There are several people in the image, includi...",...,"In the image, a man is holding a camera and lo...","In the image, a man is holding a camera, which...","The image shows a man holding a camera, with a...",The result that the people in the image are 

# 5 Tanda (ToDO)

# Tanda animal

In [15]:
for x in tqdm(indices_dataset_animal_cifar[:50]):
    respuestas=predit(2,x) # 2 porque animal
    for columna, respuesta in zip(dataset_animales_respuestas.columns, respuestas):#dataset_humano_respuestas.columns:
            #for indice_respuestas in range(0,len(respuestas)):
        dataset_animales_respuestas.loc[x, columna] = respuesta
    del respuestas

100%|██████████| 50/50 [4:06:10<00:00, 295.41s/it]  


In [17]:
dataset_animales_respuestas.to_pickle('dataset_animal_respuestas_parte1_corregida_septiembre.pkl')

# 2da Tanda animal

In [18]:
for x in tqdm(indices_dataset_animal_cifar[50:102]): ## coloco 102 por si acaso
    respuestas=predit(2,x) # 2 porque animal
    for columna, respuesta in zip(dataset_animales_respuestas.columns, respuestas):#dataset_humano_respuestas.columns:
            #for indice_respuestas in range(0,len(respuestas)):
        dataset_animales_respuestas.loc[x, columna] = respuesta
    del respuestas

100%|██████████| 52/52 [4:20:51<00:00, 300.99s/it]  


In [19]:
dataset_animales_respuestas.to_pickle('dataset_animal_respuestas_parte2_corregida_septiembre.pkl')

In [ ]:
dataset_animales_respuestas

# 3ra Tanda animal

In [15]:
for x in tqdm(indices_dataset_animal_cifar[100:150]):
    respuestas=predit(2,x) # 2 porque animal
    for columna, respuesta in zip(dataset_animales_respuestas.columns, respuestas):#dataset_humano_respuestas.columns:
            #for indice_respuestas in range(0,len(respuestas)):
        dataset_animales_respuestas.loc[x, columna] = respuesta
    del respuestas

100%|██████████| 50/50 [4:09:10<00:00, 299.01s/it]  


In [ ]:
dataset_animales_respuestas.to_pickle('dataset_animal_respuestas_parte3.pkl')

# 4ra Tanda animal (me quedo aqui para ejecutar)

In [18]:
for x in tqdm(indices_dataset_animal_cifar[150:200]):
    respuestas=predit(2,x) # 2 porque animal
    for columna, respuesta in zip(dataset_animales_respuestas.columns, respuestas):#dataset_humano_respuestas.columns:
            #for indice_respuestas in range(0,len(respuestas)):
        dataset_animales_respuestas.loc[x, columna] = respuesta
    del respuestas

100%|██████████| 50/50 [4:03:07<00:00, 291.76s/it]  


In [19]:
dataset_animales_respuestas.to_pickle('dataset_animal_respuestas_parte4.pkl')

# Tanda pruebas animal (10 muestras)

In [15]:
for x in tqdm(indices_dataset_animal_cifar[:10]):
    respuestas=predit(2,x) # 2 porque animal
    for columna, respuesta in zip(dataset_animales_respuestas.columns, respuestas):#dataset_humano_respuestas.columns:
            #for indice_respuestas in range(0,len(respuestas)):
        dataset_animales_respuestas.loc[x, columna] = respuesta
    del respuestas

100%|██████████| 10/10 [48:06<00:00, 288.66s/it]


In [19]:
dataset_animales_respuestas.to_pickle('dataset_animal_respuestas_parte_muestras.pkl')

# Tanda pruebas objeto (10 muestras)

In [16]:
for x in tqdm(indices_dataset_objeto_cifar[:10]):
    respuestas=predit(3,x) # 3 porque objeto
    for columna, respuesta in zip(dataset_objetos_respuestas.columns, respuestas):#dataset_humano_respuestas.columns:
            #for indice_respuestas in range(0,len(respuestas)):
        dataset_objetos_respuestas.loc[x, columna] = respuesta
    del respuestas

100%|██████████| 10/10 [45:38<00:00, 273.86s/it]


In [17]:
dataset_objetos_respuestas.to_pickle('dataset_objeto_respuestas_parte_muestras.pkl')

# Tanda pruebas paisaje (10 muestras) actualmento 51

In [21]:
for x in tqdm(indices_dataset_paisaje_cifar[:51]): # 51 por si acaso
    respuestas=predit(4,x) # 4 porque es paisaje
    for columna, respuesta in zip(dataset_paisajes_respuestas.columns, respuestas):#dataset_humano_respuestas.columns:
            #for indice_respuestas in range(0,len(respuestas)):
        dataset_paisajes_respuestas.loc[x, columna] = respuesta
    del respuestas

100%|██████████| 51/51 [3:38:41<00:00, 257.27s/it]  


In [22]:
dataset_paisajes_respuestas.to_pickle('dataset_paisaje_respuestas_parte_1_corregido_septiembre.pkl')

# Empezamos a trabajar con los dataset de respuestas

In [7]:

#Dataset_respuestas_Humano (ojo la parte 4 parece que tiene de mas)
dataset_humano_respuestas_parte1=unpickle('1.0 Humano/dataset_humano_respuestas_parte1_new.pkl')
dataset_humano_respuestas_parte2=unpickle('1.0 Humano/dataset_humano_respuestas_parte2_new.pkl')
dataset_humano_respuestas_parte3=unpickle('1.0 Humano/dataset_humano_respuestas_parte3_new.pkl')
dataset_humano_respuestas_parte4=unpickle('1.0 Humano/dataset_humano_respuestas_parte4_new.pkl')
dataset_humano_respuestas_parte5=unpickle('1.0 Humano/dataset_humano_respuestas_parte5_new.pkl')



#Dataset_respuestas_Animal
dataset_animal_respuestas_parte1=unpickle('2.0 Animal/dataset_animal_respuestas_parte1_corregida_septiembre.pkl')
dataset_animal_respuestas_parte2=unpickle('2.0 Animal/dataset_animal_respuestas_parte2_corregida_septiembre.pkl')
dataset_animal_respuestas_parte3=unpickle('2.0 Animal/dataset_animal_respuestas_parte3.pkl')
dataset_animal_respuestas_parte4=unpickle('2.0 Animal/dataset_animal_respuestas_parte4.pkl')
dataset_animal_respuestas_parte5=unpickle('2.0 Animal/dataset_animal_respuestas_parte5.pkl')


#Dataset_respuestas_objeto
dataset_objeto_respuestas_parte1=unpickle('3.0 objeto/dataset_objeto_respuestas_parte1.pkl')
#dataset_objeto_respuestas_parte_desconocida=unpickle('/kaggle/input/3-0dataset-categorias-preguntas-realizadas/3.0 objeto/Parece_que_con_parte5_ya_tiene_todo')
dataset_objeto_respuestas_parte2=unpickle('3.0 objeto/dataset_objeto_respuestas_parte2.pkl')
dataset_objeto_respuestas_parte3=unpickle('3.0 objeto/dataset_objeto_respuestas_parte3.pkl')
dataset_objeto_respuestas_parte4=unpickle('3.0 objeto/dataset_objeto_respuestas_parte4.pkl')
dataset_objeto_respuestas_parte5=unpickle('3.0 objeto/dataset_objeto_respuestas_parte5.pkl')



#Dataset_respuestas_paisaje
dataset_paisaje_parte1=unpickle('4.0 paisaje/dataset_paisaje_respuestas_parte1.pkl')
dataset_paisaje_parte2=unpickle('4.0 paisaje/dataset_paisaje_respuestas_parte2.pkl')
dataset_paisaje_parte3=unpickle('4.0 paisaje/dataset_paisaje_respuestas_parte3.pkl')
dataset_paisaje_parte4=unpickle('4.0 paisaje/dataset_paisaje_respuestas_parte4.pkl')
dataset_paisaje_parte5=unpickle('4.0 paisaje/dataset_paisaje_respuestas_parte5.pkl')


## comprobaciones humano

In [14]:
dataset_humano_respuestas_parte1.index

Index([  58,   65,   77,   90,   92,  122,  142,  159,  188,  190,  202,  203,
        259,  260,  267,  308,  323,  382,  401,  455,  490,  493,  562,  578,
        580,  597,  598,  614,  637,  657,  666,  692,  707,  740,  745,  767,
        795,  828,  860,  879,  885,  888,  929,  968, 1003, 1021, 1067, 1078,
       1106, 1109],
      dtype='int64')

In [15]:
dataset_humano_respuestas_parte2.index

Index([1146, 1195, 1198, 1228, 1266, 1267, 1291, 1303, 1311, 1328, 1366, 1451,
       1455, 1473, 1511, 1576, 1648, 1664, 1689, 1693, 1696, 1705, 1757, 1772,
       1778, 1784, 1798, 1804, 1837, 1852, 1853, 1860, 1861, 1870, 1871, 1873,
       1879, 1895, 1899, 1903, 1908, 1992, 2009, 2016, 2026, 2047, 2051, 2073,
       2088, 2103],
      dtype='int64')

In [16]:
dataset_humano_respuestas_parte3.index

Index([2106, 2131, 2179, 2192, 2201, 2210, 2229, 2317, 2352, 2383, 2399, 2404,
       2440, 2477, 2570, 2585, 2652, 2668, 2689, 2706, 2720, 2729, 2741, 2776,
       2827, 2832, 2858, 2894, 2902, 2904, 2925, 2933, 2954, 2999, 3011, 3012,
       3072, 3112, 3125, 3131, 3137, 3147, 3254, 3272, 3289, 3345, 3349, 3351,
       3456, 3465],
      dtype='int64')

In [17]:
dataset_humano_respuestas_parte4.index

Index([3517, 3613, 3624, 3633, 3677, 3679, 3725, 3766, 3821, 3877, 3878, 3891,
       3931, 3973, 3998, 4020, 4031, 4099, 4106, 4179, 4183, 4198, 4203, 4271,
       4331, 4366, 4382, 4429, 4451, 4481, 4487, 4489, 4500, 4515, 4555, 4568,
       4621, 4685, 4747, 4750, 4773, 4859, 4861, 4929, 4934, 4979, 4994, 5019,
       5028, 5039],
      dtype='int64')

In [18]:
dataset_humano_respuestas_parte5.index

Index([5049, 5063, 5164, 5180, 5185, 5196, 5202, 5224, 5226, 5229, 5240, 5296,
       5299, 5335, 5342, 5350, 5378, 5390, 5406, 5417, 5447, 5455, 5459, 5480,
       5487, 5564, 5582, 5584, 5617, 5650, 5661, 5674, 5738, 5776, 5784, 5802,
       5853, 5895, 5913, 5921, 5965, 5997, 6021, 6058, 6097, 6149, 6159, 6216,
       6223, 6313],
      dtype='int64')

# comprobaciones animal

In [ ]:
# parte1: 50 parte2: 100 parte3: 50 parte4: 100 parte5: 50

In [20]:
dataset_animal_respuestas_parte1.index

Index([  0,   1,   4,   9,  17,  18,  26,  28,  29,  33,  40,  41,  43,  49,
        50,  63,  68,  72,  79,  81,  82,  85,  86,  87,  91,  96,  97,  98,
        99, 103, 105, 111, 121, 124, 125, 127, 129, 136, 137, 143, 146, 147,
       150, 153, 155, 158, 160, 163, 165, 169],
      dtype='int64')

In [22]:
dataset_animal_respuestas_parte2.index

Index([170, 173, 177, 180, 182, 185, 189, 191, 192, 201, 205, 206, 210, 213,
       216, 224, 233, 238, 247, 250, 262, 265, 268, 271, 272, 278, 280, 282,
       284, 287, 292, 293, 296, 297, 301, 302, 305, 306, 310, 312, 313, 316,
       326, 327, 329, 333, 334, 335, 337, 338],
      dtype='int64')

In [30]:
dataset_animal_respuestas_parte3=dataset_animal_respuestas_parte3[dataset_animal_respuestas_parte3.index>0]

In [25]:
dataset_animal_respuestas_parte4.index

Index([531, 533, 537, 539, 543, 546, 549, 552, 556, 577, 579, 589, 593, 594,
       595, 613, 615, 616, 622, 623, 627, 628, 634, 642, 643, 645, 648, 655,
       656, 661, 662, 664, 677, 678, 683, 684, 686, 691, 695, 697, 698, 708,
       711, 712, 715, 718, 724, 726, 729, 730],
      dtype='int64')

In [27]:
dataset_animal_respuestas_parte5.index

Index([732, 733, 734, 742, 750, 757, 762, 769, 770, 777, 778, 790, 793, 794,
       797, 800, 802, 808, 812, 813, 816, 817, 818, 821, 826, 829, 832, 833,
       836, 842, 846, 848, 852, 859, 866, 869, 870, 876, 884, 886, 890, 892,
       893, 898, 903, 904, 905, 906, 910, 914],
      dtype='int64')

# comprobaciones objeto

In [41]:
dataset_objeto_respuestas_parte1.index

Index([  2,   5,   7,  16,  23,  25,  30,  35,  36,  39,  48,  52,  57,  64,
        73,  76,  93, 104, 106, 107, 109, 118, 130, 132, 134, 139, 152, 161,
       162, 168, 176, 183, 186, 195, 196, 198, 199, 211, 219, 222, 227, 229,
       234, 239, 240, 241, 244, 251, 252, 253],
      dtype='int64')

In [42]:
dataset_objeto_respuestas_parte2.index

Index([257, 263, 281, 285, 286, 289, 291, 298, 303, 304, 307, 315, 317, 319,
       325, 328, 332, 341, 350, 353, 370, 374, 378, 379, 381, 384, 385, 388,
       390, 392, 404, 405, 408, 409, 417, 421, 424, 427, 441, 450, 451, 459,
       464, 465, 468, 469, 473, 475, 477, 478],
      dtype='int64')

In [43]:
dataset_objeto_respuestas_parte3.index

Index([480, 489, 495, 496, 500, 503, 509, 518, 519, 521, 532, 535, 542, 544,
       553, 554, 555, 560, 569, 570, 571, 575, 576, 586, 587, 600, 603, 611,
       620, 630, 632, 633, 635, 636, 639, 646, 647, 654, 658, 663, 668, 670,
       673, 674, 675, 682, 688, 694, 703, 706],
      dtype='int64')

In [44]:
dataset_objeto_respuestas_parte4.index

Index([710, 713, 720, 721, 723, 736, 741, 747, 749, 754, 759, 764, 766, 773,
       779, 783, 785, 786, 789, 798, 803, 804, 806, 807, 822, 831, 839, 843,
       850, 851, 853, 858, 872, 873, 877, 887, 889, 908, 909, 912, 915, 920,
       921, 922, 926, 935, 942, 945, 946, 953],
      dtype='int64')

In [45]:
dataset_objeto_respuestas_parte5.index

Index([ 963,  964,  966,  975,  977,  979,  984,  986,  989,  994,  996,  998,
       1002, 1007, 1023, 1031, 1033, 1051, 1060, 1064, 1068, 1083, 1091, 1092,
       1098, 1105, 1114, 1122, 1126, 1127, 1130, 1132, 1137, 1138, 1152, 1154,
       1158, 1161, 1170, 1173, 1176, 1179, 1186, 1190, 1200, 1206, 1209, 1210,
       1211, 1213],
      dtype='int64')

# comprobaciones paisaje

In [ ]:
#parte 1 250, parte2: 100 parte3: 150 parte4: 200 parte5: 250 yo creo que con la parte 5 buscar porque:: porque tiene 48 preguntas que es lo correcto.

In [53]:
dataset_paisaje_parte1.index

Index([ 12,  14,  19,  21,  24,  42,  54,  67,  78,  88, 110, 115, 119, 154,
       164, 167, 204, 208, 225, 246, 274, 314, 318, 324, 330, 336, 362, 373,
       406, 416, 423, 434, 436, 456, 466, 467, 472, 479, 498, 510, 512, 515,
       516, 517, 567, 568, 574, 583, 607, 610],
      dtype='int64')

In [54]:
dataset_paisaje_parte2.index

Index([ 617,  619,  625,  649,  680,  687,  731,  738,  755,  809,  814,  819,
        834,  844,  849,  856,  864,  871,  907,  919,  923,  924,  973,  976,
        982, 1000, 1028, 1029, 1036, 1048, 1069, 1093, 1097, 1102, 1103, 1104,
       1111, 1136, 1140, 1143, 1144, 1147, 1160, 1166, 1178, 1197, 1231, 1237,
       1243, 1270],
      dtype='int64')

In [55]:
dataset_paisaje_parte3.index

Index([1295, 1296, 1298, 1332, 1341, 1371, 1378, 1402, 1432, 1434, 1445, 1450,
       1467, 1478, 1490, 1523, 1542, 1546, 1548, 1559, 1562, 1569, 1571, 1577,
       1591, 1594, 1606, 1610, 1616, 1622, 1658, 1659, 1660, 1741, 1764, 1787,
       1793, 1840, 1844, 1854, 1888, 1913, 1919, 1927, 1939, 1987, 2000, 2071,
       2098, 2099],
      dtype='int64')

In [56]:
dataset_paisaje_parte4.index

Index([2144, 2147, 2173, 2181, 2219, 2220, 2233, 2236, 2242, 2247, 2262, 2265,
       2283, 2284, 2290, 2294, 2361, 2372, 2430, 2431, 2444, 2448, 2530, 2538,
       2539, 2542, 2544, 2558, 2602, 2612, 2613, 2614, 2625, 2642, 2643, 2688,
       2701, 2737, 2753, 2755, 2781, 2806, 2834, 2844, 2862, 2878, 2883, 2920,
       2945, 2970],
      dtype='int64')

In [57]:
dataset_paisaje_parte5.index

Index([2987, 3025, 3029, 3040, 3047, 3055, 3097, 3101, 3122, 3126, 3127, 3165,
       3200, 3207, 3212, 3221, 3236, 3266, 3292, 3298, 3314, 3316, 3340, 3373,
       3390, 3407, 3425, 3438, 3477, 3486, 3488, 3500, 3507, 3510, 3522, 3555,
       3569, 3615, 3638, 3657, 3667, 3691, 3711, 3712, 3726, 3740, 3750, 3765,
       3783, 3786],
      dtype='int64')

## Tratamiento de cabeceras (eliminar los caracteres especiales de la columna)

### Para dataset respuestas humanos 

In [59]:
def convertir_cabecera(x):
    string_lista=re.findall('"([^"]*)"',x)
    return string_lista[0]

nuevas_cabeceras_1=[convertir_cabecera(x) for x in dataset_humano_respuestas_parte1.columns.tolist()]
nuevas_cabeceras_2=[convertir_cabecera(x) for x in dataset_humano_respuestas_parte2.columns.tolist()]
nuevas_cabeceras_3=[convertir_cabecera(x) for x in dataset_humano_respuestas_parte3.columns.tolist()]
nuevas_cabeceras_4=[convertir_cabecera(x) for x in dataset_humano_respuestas_parte4.columns.tolist()]
nuevas_cabeceras_5=[convertir_cabecera(x) for x in dataset_humano_respuestas_parte5.columns.tolist()]


dataset_humano_respuestas_parte1.columns=nuevas_cabeceras_1
dataset_humano_respuestas_parte2.columns=nuevas_cabeceras_2
dataset_humano_respuestas_parte3.columns=nuevas_cabeceras_3
dataset_humano_respuestas_parte4.columns=nuevas_cabeceras_4
dataset_humano_respuestas_parte5.columns=nuevas_cabeceras_5

### Para dataset respuestas animales 

In [60]:
#La parte 1 y 2 ya tienen las cabeceras actualizadas

nuevas_cabeceras_1=[convertir_cabecera(x) for x in dataset_animal_respuestas_parte1.columns.tolist()]
nuevas_cabeceras_2=[convertir_cabecera(x) for x in dataset_animal_respuestas_parte2.columns.tolist()]
nuevas_cabeceras_3=[convertir_cabecera(x) for x in dataset_animal_respuestas_parte3.columns.tolist()]
nuevas_cabeceras_4=[convertir_cabecera(x) for x in dataset_animal_respuestas_parte4.columns.tolist()]
nuevas_cabeceras_5=[convertir_cabecera(x) for x in dataset_animal_respuestas_parte5.columns.tolist()]


dataset_animal_respuestas_parte1.columns=nuevas_cabeceras_1
dataset_animal_respuestas_parte2.columns=nuevas_cabeceras_2
dataset_animal_respuestas_parte3.columns=nuevas_cabeceras_3
dataset_animal_respuestas_parte4.columns=nuevas_cabeceras_4
dataset_animal_respuestas_parte5.columns=nuevas_cabeceras_5


In [ ]:
print(dataset_animal_respuestas_parte1.columns==dataset_animal_respuestas_parte5.columns,
dataset_animal_respuestas_parte1.columns==dataset_animal_respuestas_parte2.columns,
dataset_animal_respuestas_parte1.columns==dataset_animal_respuestas_parte3.columns,
dataset_animal_respuestas_parte1.columns==dataset_animal_respuestas_parte4.columns,
dataset_animal_respuestas_parte1.columns==dataset_animal_respuestas_parte5.columns,
dataset_animal_respuestas_parte2.columns==dataset_animal_respuestas_parte1.columns,
dataset_animal_respuestas_parte2.columns==dataset_animal_respuestas_parte2.columns,
dataset_animal_respuestas_parte2.columns==dataset_animal_respuestas_parte3.columns,
dataset_animal_respuestas_parte2.columns==dataset_animal_respuestas_parte4.columns,
dataset_animal_respuestas_parte2.columns==dataset_animal_respuestas_parte5.columns,
dataset_animal_respuestas_parte3.columns==dataset_animal_respuestas_parte1.columns,
dataset_animal_respuestas_parte3.columns==dataset_animal_respuestas_parte2.columns,
dataset_animal_respuestas_parte3.columns==dataset_animal_respuestas_parte3.columns,
dataset_animal_respuestas_parte3.columns==dataset_animal_respuestas_parte4.columns,
dataset_animal_respuestas_parte3.columns==dataset_animal_respuestas_parte5.columns,
dataset_animal_respuestas_parte4.columns==dataset_animal_respuestas_parte1.columns,
dataset_animal_respuestas_parte4.columns==dataset_animal_respuestas_parte2.columns,
dataset_animal_respuestas_parte4.columns==dataset_animal_respuestas_parte3.columns,
dataset_animal_respuestas_parte4.columns==dataset_animal_respuestas_parte4.columns,
dataset_animal_respuestas_parte4.columns==dataset_animal_respuestas_parte5.columns,
dataset_animal_respuestas_parte5.columns==dataset_animal_respuestas_parte1.columns,
dataset_animal_respuestas_parte5.columns==dataset_animal_respuestas_parte2.columns,
dataset_animal_respuestas_parte5.columns==dataset_animal_respuestas_parte3.columns,
dataset_animal_respuestas_parte5.columns==dataset_animal_respuestas_parte4.columns,
dataset_animal_respuestas_parte5.columns==dataset_animal_respuestas_parte5.columns)

# resultado todo true

### Para dataset respuestas objetos 

In [62]:
nuevas_cabeceras_1=[convertir_cabecera(x) for x in dataset_objeto_respuestas_parte1.columns.tolist()]
nuevas_cabeceras_2=[convertir_cabecera(x) for x in dataset_objeto_respuestas_parte2.columns.tolist()]
nuevas_cabeceras_3=[convertir_cabecera(x) for x in dataset_objeto_respuestas_parte3.columns.tolist()]
nuevas_cabeceras_4=[convertir_cabecera(x) for x in dataset_objeto_respuestas_parte4.columns.tolist()]
nuevas_cabeceras_5=[convertir_cabecera(x) for x in dataset_objeto_respuestas_parte5.columns.tolist()]


dataset_objeto_respuestas_parte1.columns=nuevas_cabeceras_1
dataset_objeto_respuestas_parte2.columns=nuevas_cabeceras_2
dataset_objeto_respuestas_parte3.columns=nuevas_cabeceras_3
dataset_objeto_respuestas_parte4.columns=nuevas_cabeceras_4
dataset_objeto_respuestas_parte5.columns=nuevas_cabeceras_5

### Para dataset respuestas paisajes 

In [63]:
nuevas_cabeceras_1=[convertir_cabecera(x) for x in dataset_paisaje_parte1.columns.tolist()]
nuevas_cabeceras_2=[convertir_cabecera(x) for x in dataset_paisaje_parte2.columns.tolist()]
nuevas_cabeceras_3=[convertir_cabecera(x) for x in dataset_paisaje_parte3.columns.tolist()]
nuevas_cabeceras_4=[convertir_cabecera(x) for x in dataset_paisaje_parte4.columns.tolist()]
nuevas_cabeceras_5=[convertir_cabecera(x) for x in dataset_paisaje_parte5.columns.tolist()]

dataset_paisaje_parte1.columns=nuevas_cabeceras_1
dataset_paisaje_parte2.columns=nuevas_cabeceras_2
dataset_paisaje_parte3.columns=nuevas_cabeceras_3
dataset_paisaje_parte4.columns=nuevas_cabeceras_4
dataset_paisaje_parte5.columns=nuevas_cabeceras_5

## Caso paisaje el dataset solo es la parte 5, Caso objeto el dataset solo es la parte 5.

# Union dataset_respuestas y cabeceras (etiqueta_singularidad....)

In [64]:
# Union de los dataset respuestas que estan en pequeñas partes en uno solo

dataset_humano_respuestas= pd.concat([dataset_humano_respuestas_parte1,
                                        dataset_humano_respuestas_parte2,
                                        dataset_humano_respuestas_parte3,
                                        dataset_humano_respuestas_parte4,
                                        dataset_humano_respuestas_parte5], ignore_index=False)

dataset_animal_respuestas=pd.concat([dataset_animal_respuestas_parte1,
                                    dataset_animal_respuestas_parte2,
                                    dataset_animal_respuestas_parte3,
                                    dataset_animal_respuestas_parte4,
                                    dataset_animal_respuestas_parte5], ignore_index=False)


dataset_objeto_respuestas= pd.concat([dataset_objeto_respuestas_parte1,
                                        dataset_objeto_respuestas_parte2,
                                        dataset_objeto_respuestas_parte3,
                                        dataset_objeto_respuestas_parte4,
                                        dataset_objeto_respuestas_parte5], ignore_index=False)


dataset_paisaje_respuestas= pd.concat([dataset_paisaje_parte1,
                                        dataset_paisaje_parte2,
                                        dataset_paisaje_parte3,
                                        dataset_paisaje_parte4,
                                        dataset_paisaje_parte5], ignore_index=False)

# para este caso no es necesario concatenar
#dataset_objeto_respuestas=dataset_objeto_respuestas_parte5


# para este caso no es necesario concatenar
#dataset_paisaje_respuestas=dataset_paisaje_parte5

In [65]:
# para evitar problemas de clave en el join de tablas

dataset_humano_cifar.loc[:,'indice_2']=dataset_humano_cifar.index
dataset_humano_respuestas.loc[:,'indice_2']=dataset_humano_respuestas.index

dataset_animal_cifar.loc[:,'indice_2']=dataset_animal_cifar.index
dataset_animal_respuestas.loc[:,'indice_2']=dataset_animal_respuestas.index

dataset_objeto_cifar.loc[:,'indice_2']=dataset_objeto_cifar.index
dataset_objeto_respuestas.loc[:,'indice_2']=dataset_objeto_respuestas.index

dataset_paisaje_cifar.loc[:,'indice_2']=dataset_paisaje_cifar.index
dataset_paisaje_respuestas.loc[:,'indice_2']=dataset_paisaje_respuestas.index


In [10]:
# funcion sql
def sql_query(query):
    q=sql.sqldf
    consulta=q(query,globals())
    return consulta

In [67]:
query_humano='''
SELECT dataset_humano_cifar."b'data'", dataset_humano_cifar.etiqueta, dataset_humano_cifar.singularidad, dataset_humano_cifar.categoria,  dataset_humano_respuestas.* FROM dataset_humano_cifar, dataset_humano_respuestas
WHERE dataset_humano_cifar.indice_2=dataset_humano_respuestas.indice_2
'''

query_animal='''
SELECT dataset_animal_cifar."b'data'", dataset_animal_cifar.etiqueta, dataset_animal_cifar.singularidad, dataset_animal_cifar.categoria,  dataset_animal_respuestas.* FROM dataset_animal_cifar, dataset_animal_respuestas
WHERE dataset_animal_cifar.indice_2=dataset_animal_respuestas.indice_2
'''

query_objeto='''
SELECT dataset_objeto_cifar."b'data'", dataset_objeto_cifar.etiqueta, dataset_objeto_cifar.singularidad, dataset_objeto_cifar.categoria,  dataset_objeto_respuestas.* FROM dataset_objeto_cifar, dataset_objeto_respuestas
WHERE dataset_objeto_cifar.indice_2=dataset_objeto_respuestas.indice_2
'''

query_paisaje='''
SELECT dataset_paisaje_cifar."b'data'", dataset_paisaje_cifar.etiqueta, dataset_paisaje_cifar.singularidad, dataset_paisaje_cifar.categoria,  dataset_paisaje_respuestas.* FROM dataset_paisaje_cifar, dataset_paisaje_respuestas
WHERE dataset_paisaje_cifar.indice_2=dataset_paisaje_respuestas.indice_2
'''

dataset_humano_respuestas_completo=sql_query(query_humano)
dataset_animal_respuestas_completo=sql_query(query_animal)
dataset_objeto_respuestas_completo=sql_query(query_objeto)
dataset_paisaje_respuestas_completo=sql_query(query_paisaje)

In [68]:
indices_singular_humano=dataset_humano_respuestas_completo[dataset_humano_respuestas_completo['singularidad']=='singular']['indice_2'].tolist()
indices_singular_animal=dataset_animal_respuestas_completo[dataset_animal_respuestas_completo['singularidad']=='singular']['indice_2'].tolist()
indices_singular_objeto=dataset_objeto_respuestas_completo[dataset_objeto_respuestas_completo['singularidad']=='singular']['indice_2'].tolist()
indices_singular_paisaje=dataset_paisaje_respuestas_completo[dataset_paisaje_respuestas_completo['singularidad']=='singular']['indice_2'].tolist()

In [69]:
# resetear index
dataset_humano_respuestas_completo.set_index(dataset_humano_respuestas_completo['indice_2'],inplace=True)
dataset_humano_respuestas_completo = dataset_humano_respuestas_completo.rename(columns={'indice_2': 'indice'})

# resetear index
dataset_animal_respuestas_completo.set_index(dataset_animal_respuestas_completo['indice_2'],inplace=True)
dataset_animal_respuestas_completo = dataset_animal_respuestas_completo.rename(columns={'indice_2': 'indice'})

# resetear index
dataset_objeto_respuestas_completo.set_index(dataset_objeto_respuestas_completo['indice_2'],inplace=True)
dataset_objeto_respuestas_completo = dataset_objeto_respuestas_completo.rename(columns={'indice_2': 'indice'})

# resetear index
dataset_paisaje_respuestas_completo.set_index(dataset_paisaje_respuestas_completo['indice_2'],inplace=True)
dataset_paisaje_respuestas_completo = dataset_paisaje_respuestas_completo.rename(columns={'indice_2': 'indice'})

In [70]:
# Colocamos 0000 para identificar que es nulo, esto porque np.nan uego da problemas para manipulacion explotacion.

#columnas plularidad humano
dataset_humano_respuestas_completo.loc[indices_singular_humano,[' have similar colors?','Are  far away from each other?','Do  interact with each other?']]=0000

#columnas plularidad animal
dataset_animal_respuestas_completo.loc[indices_singular_animal,[' have similar colors?','Are  far away from each other?','Do  interact with each other?']]=0000


#columnas plularidad objeto
dataset_objeto_respuestas_completo.loc[indices_singular_objeto,[' have similar colors?', "Are  far away from each other?"]]=0000


#columnas plularidad paisaje
dataset_paisaje_respuestas_completo.loc[indices_singular_paisaje,' have similar colors?']=0000

# Filtro numero 1

## Para obtener las categorias y singularidades presentes a trabajar

In [71]:
#agrupacion por etiqueta y singularidad
# AQUI SOLO HAY HUMANOS CREO QUE SERIA PARA EL RESTO DE DATASET TAMBIEN
# SERIA AGREGAR LOS UNICOS DEL RESTO DE DATASETS
query_unicos_humanos= '''
SELECT DISTINCT(ETIQUETA), SINGULARIDAD FROM dataset_humano_respuestas_completo
'''

query_unicos_animal= '''
SELECT DISTINCT(ETIQUETA), SINGULARIDAD FROM dataset_animal_respuestas_completo
'''

query_unicos_objeto= '''
SELECT DISTINCT(ETIQUETA), SINGULARIDAD FROM dataset_objeto_respuestas_completo
'''

query_unicos_paisaje='''
SELECT DISTINCT(ETIQUETA), SINGULARIDAD FROM dataset_paisaje_respuestas_completo
'''

unicos_humanos= sql_query(query_unicos_humanos)
unicos_animal= sql_query(query_unicos_animal)
unicos_objetos= sql_query(query_unicos_objeto)
unicos_paisajes= sql_query(query_unicos_paisaje)

In [72]:
unicos_humanos

,etiqueta,singularidad
0,woman,singular
1,girl,singular
2,man,singular
3,baby,singular


In [73]:
unicos_animal

,etiqueta,singularidad
0,cattle,plural
1,dinosaur,singular
2,aquarium_fish,singular
3,elephant,singular
4,wolf,singular
5,squirrel,singular
6,possum,singular
7,leopard,singular
8,rabbit,singular
9,snake,singular


In [74]:
unicos_objetos

,etiqueta,singularidad
0,apple,singular
1,telephone,singular
2,cup,singular
3,bicycle,singular
4,television,singular
5,table,singular
6,clock,singular
7,streetcar,singular
8,tractor,singular
9,chair,singular


In [75]:
unicos_paisajes

,etiqueta,singularidad
0,sunflower,singular
1,sea,singular
2,pine_tree,singular
3,mountain,singular
4,forest,singular
5,orchid,singular


# para tener los subsatadets por etiqueta y singularidad

## Caso Humanos

In [76]:
subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_humanos)):
    #AQUI TAMBIEN AÑADIR LOS OTROS DATASETS
    subdataset_para_comparativa_filtro_1.append(dataset_humano_respuestas_completo[(dataset_humano_respuestas_completo['etiqueta']==unicos_humanos['etiqueta'][i]) &
                                   (dataset_humano_respuestas_completo['singularidad']==unicos_humanos['singularidad'][i])])

# funciones de metricas

In [3]:
%pip install textdistance

  Using cached textdistance-4.6.3-py3-none-any.whl.metadata (18 kB)
Using cached textdistance-4.6.3-py3-none-any.whl (31 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import textdistance

# Levestehin
def similitud_metrica_levestehin(texto_query,texto_iteracion):
    texto_query=str(texto_query)
    texto_iteracion= str(texto_iteracion)
    if texto_query!='0':
        similitud=textdistance.levenshtein(texto_query,texto_iteracion)
        return similitud
    else:
        return 6666


    
# overlap coefficient
def similitud_metrica_overlap(texto_query,texto_iteracion):
    texto_query=str(texto_query)
    texto_iteracion= str(texto_iteracion)
    if texto_query!='0':
        similitud=textdistance.overlap(texto_query,texto_iteracion)
        return similitud
    else:
        return 6666


    
# cosine similarity
def similitud_metrica_cosine(texto_query,texto_iteracion):
    texto_query=str(texto_query)
    texto_iteracion= str(texto_iteracion)
    if texto_query!='0':
        similitud=textdistance.cosine(texto_query,texto_iteracion)
        return similitud
    else:
        return 6666
    

    
# Jaccard index
def similitud_metrica_jaccard(texto_query,texto_iteracion):
    texto_query=str(texto_query)
    texto_iteracion= str(texto_iteracion)
    if texto_query!='0':
        similitud=textdistance.jaccard(texto_query,texto_iteracion)
        return similitud
    else:
        return 6666


    
# bag distance 0 significa que es igual
def similitud_metrica_bag(texto_query,texto_iteracion):
    texto_query=str(texto_query)
    texto_iteracion= str(texto_iteracion)
    if texto_query!='0':
        similitud=textdistance.bag(texto_query,texto_iteracion)
        return similitud
    else:
        return 6666

In [3]:
web_model = WebBertSimilarity(device='cuda',batch_size=10) #defaults to GPU prediction

In [1]:


web_model = WebBertSimilarity(device='cpu', batch_size=10) #defaults to GPU prediction

clinical_model = ClinicalBertSimilarity(device='cuda', batch_size=10) #defaults to GPU prediction

#web_model.predict([("She won an olympic gold medal","The women is an olympic champion")])

/home/angel/miniconda3/envs/TFM_2/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


100%|██████████| 401555686/401555686 [07:37<00:00, 877872.77B/s] 


In [5]:
# funcion metrica de similitud_1 semantica
def similitud_metrica_websimilarity(texto_query,texto_iteracion):
    texto_query=str(texto_query)
    texto_iteracion=str(texto_iteracion)
    if texto_query!='0':
        similitud=web_model.predict([(texto_query,texto_iteracion)])
        return float(similitud[0]) # va del 0 al 5
    else:
        return 6666

# estudiar el multitrheads porque es muy lento

In [82]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# Suponiendo que similitud_metrica_levestehin y subdataset_para_comparativa_filtro_1 están definidos

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_levestehin(texto_query, texto_iteracion)
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # Crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # Recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)

# Ahora lista_comparacion contiene todos los resultados


62it [00:04, 14.46it/s]
67it [00:04, 13.86it/s]
61it [00:03, 15.40it/s]
60it [00:04, 14.47it/s]


In [83]:
df_resultado=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion'])


In [84]:
df_resultado.to_csv('salida_prueba_humano_3.csv', index=False)

## Caso animales

In [85]:
subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_animal)):
    
    subdataset_para_comparativa_filtro_1.append(dataset_animal_respuestas_completo[(dataset_animal_respuestas_completo['etiqueta']==unicos_animal['etiqueta'][i]) &
                                   (dataset_animal_respuestas_completo['singularidad']==unicos_animal['singularidad'][i])])

In [86]:
# Suponiendo que similitud_metrica_levestehin y subdataset_para_comparativa_filtro_1 están definidos

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_levestehin(texto_query, texto_iteracion)
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)

13it [00:00, 57.53it/s]
10it [00:00, 72.26it/s]
7it [00:00, 83.40it/s]
10it [00:00, 68.09it/s]
12it [00:00, 58.95it/s]
8it [00:00, 85.38it/s]
14it [00:00, 52.94it/s]
10it [00:00, 64.96it/s]
15it [00:00, 51.08it/s]
8it [00:00, 82.33it/s]
11it [00:00, 64.77it/s]
9it [00:00, 72.65it/s]
9it [00:00, 76.79it/s]
9it [00:00, 74.93it/s]
7it [00:00, 90.32it/s]
8it [00:00, 81.83it/s]
17it [00:00, 48.16it/s]
8it [00:00, 91.12it/s]
4it [00:00, 129.78it/s]
6it [00:00, 105.95it/s]
10it [00:00, 70.41it/s]
6it [00:00, 103.33it/s]
7it [00:00, 94.16it/s]
11it [00:00, 68.25it/s]
10it [00:00, 72.60it/s]
6it [00:00, 104.79it/s]
5it [00:00, 117.30it/s]


In [87]:
df_resultado_animales=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion'])

In [88]:
df_resultado_animales.to_csv('salida_prueba_animal_3.csv', index=False)

## Caso objetos

In [89]:
subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_objetos)):
    
    subdataset_para_comparativa_filtro_1.append(dataset_objeto_respuestas_completo[(dataset_objeto_respuestas_completo['etiqueta']==unicos_objetos['etiqueta'][i]) &
                                   (dataset_objeto_respuestas_completo['singularidad']==unicos_objetos['singularidad'][i])])

In [90]:
# Suponiendo que similitud_metrica_levestehin y subdataset_para_comparativa_filtro_1 están definidos

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_levestehin(texto_query, texto_iteracion)
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)

12it [00:00, 66.81it/s]
9it [00:00, 84.52it/s]
11it [00:00, 71.84it/s]
14it [00:00, 58.97it/s]
15it [00:00, 56.13it/s]
18it [00:00, 45.69it/s]
9it [00:00, 76.99it/s]
12it [00:00, 28.95it/s]
16it [00:00, 51.73it/s]
12it [00:00, 70.52it/s]
14it [00:00, 53.93it/s]
10it [00:00, 73.38it/s]
10it [00:00, 79.45it/s]
16it [00:00, 53.48it/s]
12it [00:00, 68.26it/s]
12it [00:00, 68.43it/s]
10it [00:00, 70.94it/s]
5it [00:00, 121.49it/s]
9it [00:00, 80.48it/s]
16it [00:00, 53.33it/s]
8it [00:00, 97.19it/s]


In [91]:
df_resultado_objeto=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion'])

In [92]:
df_resultado_objeto.to_csv('salida_prueba_objeto_3.csv', index=False)

## Caso paisajes

In [93]:
subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_paisajes)):
    
    subdataset_para_comparativa_filtro_1.append(dataset_paisaje_respuestas_completo[(dataset_paisaje_respuestas_completo['etiqueta']==unicos_paisajes['etiqueta'][i]) &
                                   (dataset_paisaje_respuestas_completo['singularidad']==unicos_paisajes['singularidad'][i])])

In [94]:
# Suponiendo que similitud_metrica_levestehin y subdataset_para_comparativa_filtro_1 están definidos

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_levestehin(texto_query, texto_iteracion)
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)

35it [00:01, 25.11it/s]
52it [00:03, 17.13it/s]
39it [00:01, 22.59it/s]
39it [00:01, 21.33it/s]
48it [00:02, 20.47it/s]
37it [00:01, 23.37it/s]


In [95]:
df_resultado_paisaje=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion'])

In [96]:
df_resultado_paisaje.to_csv('salida_prueba_paisaje_3.csv', index=False)

# Ahora empezamos el procedimiento con la similitud overlap

# Caso humano overlap

In [97]:
# para trabajar con los cercanos dividimos en sub dataset por etiqueta

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_humanos)):
    #AQUI TAMBIEN AÑADIR LOS OTROS DATASETS
    subdataset_para_comparativa_filtro_1.append(dataset_humano_respuestas_completo[(dataset_humano_respuestas_completo['etiqueta']==unicos_humanos['etiqueta'][i]) &
                                   (dataset_humano_respuestas_completo['singularidad']==unicos_humanos['singularidad'][i])])
    


# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_overlap(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # Crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # Recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)


# aqui guardamos la metrica

df_resultado_humano=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion'])


df_resultado_humano.to_csv('salida_humano_overlap_3.csv', index=False)


62it [00:08,  7.53it/s]
67it [00:09,  7.02it/s]
61it [00:08,  7.32it/s]
60it [00:08,  7.43it/s]


# Caso animales overlap

In [98]:
# para trabajar con los cercanos dividimos en sub dataset por etiqueta

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_animal)):

    subdataset_para_comparativa_filtro_1.append(dataset_animal_respuestas_completo[(dataset_animal_respuestas_completo['etiqueta']==unicos_animal['etiqueta'][i]) &
                                   (dataset_animal_respuestas_completo['singularidad']==unicos_animal['singularidad'][i])])
    


# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_overlap(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # Crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # Recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)


# aqui guardamos la metrica

df_resultado_animales=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion'])


df_resultado_animales.to_csv('salida_animal_overlap_3.csv', index=False)

13it [00:00, 31.21it/s]
10it [00:00, 38.78it/s]
7it [00:00, 60.01it/s]
10it [00:00, 42.20it/s]
12it [00:00, 34.03it/s]
8it [00:00, 47.08it/s]
14it [00:00, 27.63it/s]
10it [00:00, 37.57it/s]
15it [00:00, 26.70it/s]
8it [00:00, 46.26it/s]
11it [00:00, 34.62it/s]
9it [00:00, 41.98it/s]
9it [00:00, 40.59it/s]
9it [00:00, 18.62it/s]
7it [00:00, 51.15it/s]
8it [00:00, 46.58it/s]
17it [00:00, 23.49it/s]
8it [00:00, 46.04it/s]
4it [00:00, 83.40it/s]
6it [00:00, 61.74it/s]
10it [00:00, 37.55it/s]
6it [00:00, 60.91it/s]
7it [00:00, 52.63it/s]
11it [00:00, 34.19it/s]
10it [00:00, 38.66it/s]
6it [00:00, 60.52it/s]
5it [00:00, 68.35it/s]


# Caso objeto overlap 

In [99]:
# para trabajar con los cercanos dividimos en sub dataset por etiqueta

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_objetos)):
    # modificar dataset
    subdataset_para_comparativa_filtro_1.append(dataset_objeto_respuestas_completo[(dataset_objeto_respuestas_completo['etiqueta']==unicos_objetos['etiqueta'][i]) &
                                   (dataset_objeto_respuestas_completo['singularidad']==unicos_objetos['singularidad'][i])])
    

# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_overlap(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # eecoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)


# aqui guardamos la metrica

df_resultado_objeto=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_objeto.to_csv('salida_objeto_overlap_3.csv', index=False) # modificar

12it [00:00, 32.10it/s]
9it [00:00, 46.05it/s]
11it [00:00, 38.92it/s]
14it [00:00, 30.15it/s]
15it [00:00, 27.69it/s]
18it [00:00, 23.94it/s]
9it [00:00, 45.53it/s]
12it [00:00, 33.34it/s]
16it [00:00, 25.85it/s]
12it [00:00, 36.38it/s]
14it [00:00, 28.97it/s]
10it [00:00, 39.05it/s]
10it [00:00, 41.77it/s]
16it [00:00, 26.77it/s]
12it [00:00, 33.52it/s]
12it [00:00, 34.60it/s]
10it [00:00, 40.80it/s]
5it [00:00, 76.12it/s]
9it [00:00, 44.76it/s]
16it [00:00, 26.98it/s]
8it [00:00, 50.72it/s]


# Caso paisaje overlap

In [100]:
# para trabajar con los cercanos dividimos en sub dataset por etiqueta

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_paisajes)):
    # modificar dataset
    subdataset_para_comparativa_filtro_1.append(dataset_paisaje_respuestas_completo[(dataset_paisaje_respuestas_completo['etiqueta']==unicos_paisajes['etiqueta'][i]) &
                                   (dataset_paisaje_respuestas_completo['singularidad']==unicos_paisajes['singularidad'][i])])
    

# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_overlap(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # eecoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)


# aqui guardamos la metrica

df_resultado_paisaje=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_paisaje.to_csv('salida_paisaje_overlap_3.csv', index=False) # modificar

0it [00:00, ?it/s]

35it [00:02, 13.72it/s]
52it [00:05,  9.35it/s]
39it [00:02, 13.45it/s]
39it [00:03, 12.61it/s]
48it [00:04, 10.33it/s]
37it [00:02, 13.60it/s]


# Ahora empezamos el procedimiento con la similitud cosine

# Caso humano cosine

In [101]:
# para trabajar con los cercanos dividimos en sub dataset por etiqueta

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_humanos)):
    #AQUI TAMBIEN AÑADIR LOS OTROS DATASETS
    subdataset_para_comparativa_filtro_1.append(dataset_humano_respuestas_completo[(dataset_humano_respuestas_completo['etiqueta']==unicos_humanos['etiqueta'][i]) &
                                   (dataset_humano_respuestas_completo['singularidad']==unicos_humanos['singularidad'][i])])
    


# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_cosine(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # Crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # Recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)


# aqui guardamos la metrica

df_resultado_humano=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion'])


df_resultado_humano.to_csv('salida_humano_cosine_3.csv', index=False)


62it [00:09,  6.51it/s]
67it [00:10,  6.37it/s]
61it [00:08,  7.42it/s]
60it [00:08,  7.07it/s]


# Caso animal cosine

In [102]:
# para trabajar con los cercanos dividimos en sub dataset por etiqueta

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_animal)):

    subdataset_para_comparativa_filtro_1.append(dataset_animal_respuestas_completo[(dataset_animal_respuestas_completo['etiqueta']==unicos_animal['etiqueta'][i]) &
                                   (dataset_animal_respuestas_completo['singularidad']==unicos_animal['singularidad'][i])])
    


# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_cosine(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # Crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # Recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)


# aqui guardamos la metrica

df_resultado_animales=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_animales.to_csv('salida_animal_cosine_3.csv', index=False) # modificar

0it [00:00, ?it/s]

13it [00:00, 30.28it/s]
10it [00:00, 40.91it/s]
7it [00:00, 56.27it/s]
10it [00:00, 38.45it/s]
12it [00:00, 31.63it/s]
8it [00:00, 45.66it/s]
14it [00:00, 27.26it/s]
10it [00:00, 35.00it/s]
15it [00:00, 24.75it/s]
8it [00:00, 43.08it/s]
11it [00:00, 33.71it/s]
9it [00:00, 40.31it/s]
9it [00:00, 41.55it/s]
9it [00:00, 40.21it/s]
7it [00:00, 50.70it/s]
8it [00:00, 44.13it/s]
17it [00:00, 22.86it/s]
8it [00:00, 44.04it/s]
4it [00:00, 79.51it/s]
6it [00:00, 53.01it/s]
10it [00:00, 33.15it/s]
6it [00:00, 50.58it/s]
7it [00:00, 44.13it/s]
11it [00:00, 30.88it/s]
10it [00:00, 35.35it/s]
6it [00:00, 53.54it/s]
5it [00:00, 64.50it/s]


# Caso objeto cosine

In [103]:
# para trabajar con los cercanos dividimos en sub dataset por etiqueta

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_objetos)):
    # modificar dataset
    subdataset_para_comparativa_filtro_1.append(dataset_objeto_respuestas_completo[(dataset_objeto_respuestas_completo['etiqueta']==unicos_objetos['etiqueta'][i]) &
                                   (dataset_objeto_respuestas_completo['singularidad']==unicos_objetos['singularidad'][i])])
    

# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_cosine(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # eecoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)


# aqui guardamos la metrica

df_resultado_objeto=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_objeto.to_csv('salida_objeto_cosine_3.csv', index=False) # modificar

12it [00:00, 32.56it/s]
9it [00:00, 44.03it/s]
11it [00:00, 36.34it/s]
14it [00:00, 28.62it/s]
15it [00:00, 26.65it/s]
18it [00:00, 21.32it/s]
9it [00:00, 40.58it/s]
12it [00:00, 31.76it/s]
16it [00:00, 24.61it/s]
12it [00:00, 32.78it/s]
14it [00:00, 18.50it/s]
10it [00:00, 41.76it/s]
10it [00:00, 46.62it/s]
16it [00:00, 27.67it/s]
12it [00:00, 34.11it/s]
12it [00:00, 35.45it/s]
10it [00:00, 36.46it/s]
5it [00:00, 75.66it/s]
9it [00:00, 44.30it/s]
16it [00:00, 27.75it/s]
8it [00:00, 49.46it/s]


# Caso paisaje cosine

In [104]:
# para trabajar con los cercanos dividimos en sub dataset por etiqueta

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_paisajes)):
    # modificar dataset
    subdataset_para_comparativa_filtro_1.append(dataset_paisaje_respuestas_completo[(dataset_paisaje_respuestas_completo['etiqueta']==unicos_paisajes['etiqueta'][i]) &
                                   (dataset_paisaje_respuestas_completo['singularidad']==unicos_paisajes['singularidad'][i])])
    

# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_cosine(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # eecoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)


# aqui guardamos la metrica

df_resultado_paisaje=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_paisaje.to_csv('salida_paisaje_cosine_3.csv', index=False) # modificar

35it [00:02, 12.78it/s]
52it [00:05,  9.45it/s]
39it [00:03, 12.53it/s]
39it [00:03, 12.25it/s]
48it [00:04, 10.04it/s]
37it [00:02, 13.53it/s]


# Ahora empezamos el procedimiento con la similitud jaccard

# Caso humano jaccard

In [105]:
# para trabajar con los cercanos dividimos en sub dataset por etiqueta

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_humanos)):
    #AQUI TAMBIEN AÑADIR LOS OTROS DATASETS
    subdataset_para_comparativa_filtro_1.append(dataset_humano_respuestas_completo[(dataset_humano_respuestas_completo['etiqueta']==unicos_humanos['etiqueta'][i]) &
                                   (dataset_humano_respuestas_completo['singularidad']==unicos_humanos['singularidad'][i])])
    


# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_jaccard(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # Crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # Recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)


# aqui guardamos la metrica

df_resultado_humano=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion'])


df_resultado_humano.to_csv('salida_humano_jaccard_3.csv', index=False)


0it [00:00, ?it/s]

62it [00:10,  6.17it/s]
67it [00:11,  6.01it/s]
61it [00:08,  7.05it/s]
60it [00:08,  6.93it/s]


# Caso animal jaccard

In [106]:
# para trabajar con los cercanos dividimos en sub dataset por etiqueta

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_animal)):

    subdataset_para_comparativa_filtro_1.append(dataset_animal_respuestas_completo[(dataset_animal_respuestas_completo['etiqueta']==unicos_animal['etiqueta'][i]) &
                                   (dataset_animal_respuestas_completo['singularidad']==unicos_animal['singularidad'][i])])
    


# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_jaccard(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # Crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # Recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)


# aqui guardamos la metrica

df_resultado_animales=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_animales.to_csv('salida_animal_jaccard_3.csv', index=False) # modificar

13it [00:00, 28.83it/s]
10it [00:00, 38.89it/s]
7it [00:00, 55.22it/s]
10it [00:00, 38.58it/s]
12it [00:00, 29.71it/s]
8it [00:00, 44.17it/s]
14it [00:00, 25.52it/s]
10it [00:00, 35.38it/s]
15it [00:00, 24.73it/s]
8it [00:00, 42.88it/s]
11it [00:00, 32.02it/s]
9it [00:00, 39.32it/s]
9it [00:00, 39.73it/s]
9it [00:00, 37.72it/s]
7it [00:00, 49.70it/s]
8it [00:00, 42.75it/s]
17it [00:00, 21.84it/s]
8it [00:00, 44.86it/s]
4it [00:00, 82.27it/s]
6it [00:00, 59.52it/s]
10it [00:00, 34.77it/s]
6it [00:00, 56.17it/s]
7it [00:00, 49.26it/s]
11it [00:00, 33.09it/s]
10it [00:00, 35.76it/s]
6it [00:00, 57.64it/s]
5it [00:00, 64.65it/s]


# Caso objeto jaccard

In [107]:
# para trabajar con los cercanos dividimos en sub dataset por etiqueta

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_objetos)):
    # modificar dataset
    subdataset_para_comparativa_filtro_1.append(dataset_objeto_respuestas_completo[(dataset_objeto_respuestas_completo['etiqueta']==unicos_objetos['etiqueta'][i]) &
                                   (dataset_objeto_respuestas_completo['singularidad']==unicos_objetos['singularidad'][i])])
    

# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_jaccard(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # eecoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)


# aqui guardamos la metrica

df_resultado_objeto=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_objeto.to_csv('salida_objeto_jaccard_3.csv', index=False) # modificar

12it [00:00, 31.92it/s]
9it [00:00, 42.64it/s]
11it [00:00, 35.54it/s]
14it [00:00, 27.91it/s]
15it [00:00, 17.21it/s]
18it [00:00, 21.37it/s]
9it [00:00, 43.89it/s]
12it [00:00, 30.90it/s]
16it [00:00, 24.16it/s]
12it [00:00, 32.48it/s]
14it [00:00, 27.49it/s]
10it [00:00, 35.77it/s]
10it [00:00, 38.80it/s]
16it [00:00, 24.95it/s]
12it [00:00, 31.04it/s]
12it [00:00, 33.48it/s]
10it [00:00, 38.41it/s]
5it [00:00, 73.11it/s]
9it [00:00, 42.33it/s]
16it [00:00, 24.75it/s]
8it [00:00, 45.88it/s]


# Caso paisaje jaccard

In [108]:
# para trabajar con los cercanos dividimos en sub dataset por etiqueta

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_paisajes)):
    # modificar dataset
    subdataset_para_comparativa_filtro_1.append(dataset_paisaje_respuestas_completo[(dataset_paisaje_respuestas_completo['etiqueta']==unicos_paisajes['etiqueta'][i]) &
                                   (dataset_paisaje_respuestas_completo['singularidad']==unicos_paisajes['singularidad'][i])])
    

# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_jaccard(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # eecoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)


# aqui guardamos la metrica

df_resultado_paisaje=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_paisaje.to_csv('salida_paisaje_jaccard_3.csv', index=False) # modificar

35it [00:02, 13.51it/s]
52it [00:05,  8.69it/s]
39it [00:03, 11.43it/s]
39it [00:03, 10.92it/s]
48it [00:05,  8.84it/s]
37it [00:02, 13.08it/s]


# Ahora empezamos el procedimiento con la similitud bag

# Caso humano bag

In [109]:
# para trabajar con los cercanos dividimos en sub dataset por etiqueta

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_humanos)):
    #AQUI TAMBIEN AÑADIR LOS OTROS DATASETS
    subdataset_para_comparativa_filtro_1.append(dataset_humano_respuestas_completo[(dataset_humano_respuestas_completo['etiqueta']==unicos_humanos['etiqueta'][i]) &
                                   (dataset_humano_respuestas_completo['singularidad']==unicos_humanos['singularidad'][i])])
    


# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_bag(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # Crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # Recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)


# aqui guardamos la metrica

df_resultado_humano=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion'])


df_resultado_humano.to_csv('salida_humano_bag_3.csv', index=False)

62it [00:10,  5.88it/s]
67it [00:11,  5.80it/s]
61it [00:09,  6.41it/s]
60it [00:09,  6.61it/s]


# Caso animal bag

In [110]:
# para trabajar con los cercanos dividimos en sub dataset por etiqueta

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_animal)):

    subdataset_para_comparativa_filtro_1.append(dataset_animal_respuestas_completo[(dataset_animal_respuestas_completo['etiqueta']==unicos_animal['etiqueta'][i]) &
                                   (dataset_animal_respuestas_completo['singularidad']==unicos_animal['singularidad'][i])])
    


# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_bag(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # Crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # Recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)


# aqui guardamos la metrica

df_resultado_animales=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_animales.to_csv('salida_animal_bag_3.csv', index=False) # modificar

13it [00:00, 26.40it/s]
10it [00:00, 35.08it/s]
7it [00:00, 50.90it/s]
10it [00:00, 34.31it/s]
12it [00:00, 27.12it/s]
8it [00:00, 41.94it/s]
14it [00:00, 23.63it/s]
10it [00:00, 32.16it/s]
15it [00:00, 22.74it/s]
8it [00:00, 40.70it/s]
11it [00:00, 30.58it/s]
9it [00:00, 36.14it/s]
9it [00:00, 36.54it/s]
9it [00:00, 35.89it/s]
7it [00:00, 44.27it/s]
8it [00:00, 40.81it/s]
17it [00:00, 20.24it/s]
8it [00:00, 40.01it/s]
4it [00:00, 12.66it/s]
6it [00:00, 53.28it/s]
10it [00:00, 32.40it/s]
6it [00:00, 52.67it/s]
7it [00:00, 44.43it/s]
11it [00:00, 29.89it/s]
10it [00:00, 32.28it/s]
6it [00:00, 52.14it/s]
5it [00:00, 61.88it/s]


# Caso objeto bag

In [111]:
# para trabajar con los cercanos dividimos en sub dataset por etiqueta

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_objetos)):
    # modificar dataset
    subdataset_para_comparativa_filtro_1.append(dataset_objeto_respuestas_completo[(dataset_objeto_respuestas_completo['etiqueta']==unicos_objetos['etiqueta'][i]) &
                                   (dataset_objeto_respuestas_completo['singularidad']==unicos_objetos['singularidad'][i])])
    

# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_bag(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # eecoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)


# aqui guardamos la metrica

df_resultado_objeto=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_objeto.to_csv('salida_objeto_bag_3.csv', index=False) # modificar

0it [00:00, ?it/s]

12it [00:00, 28.67it/s]
9it [00:00, 39.97it/s]
11it [00:00, 32.90it/s]
14it [00:00, 25.83it/s]
15it [00:00, 23.04it/s]
18it [00:00, 20.02it/s]
9it [00:00, 38.23it/s]
12it [00:00, 27.91it/s]
16it [00:00, 22.44it/s]
12it [00:00, 30.76it/s]
14it [00:00, 25.46it/s]
10it [00:00, 35.38it/s]
10it [00:00, 36.26it/s]
16it [00:00, 23.25it/s]
12it [00:00, 29.01it/s]
12it [00:00, 30.29it/s]
10it [00:00, 34.53it/s]
5it [00:00, 67.71it/s]
9it [00:00, 39.65it/s]
16it [00:00, 26.02it/s]
8it [00:00, 51.52it/s]


# Dataset paisaje bag

In [112]:
# para trabajar con los cercanos dividimos en sub dataset por etiqueta

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_paisajes)):
    # modificar dataset
    subdataset_para_comparativa_filtro_1.append(dataset_paisaje_respuestas_completo[(dataset_paisaje_respuestas_completo['etiqueta']==unicos_paisajes['etiqueta'][i]) &
                                   (dataset_paisaje_respuestas_completo['singularidad']==unicos_paisajes['singularidad'][i])])
    

# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_bag(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # eecoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)


# aqui guardamos la metrica

df_resultado_paisaje=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_paisaje.to_csv('salida_paisaje_bag_3.csv', index=False) # modificar


35it [00:02, 12.45it/s]
52it [00:07,  7.39it/s]
39it [00:03, 10.21it/s]
39it [00:04,  9.57it/s]
48it [00:05,  8.78it/s]
37it [00:03, 12.00it/s]


In [113]:
### voy a guardar los dataset completos
dataset_humano_respuestas_completo.to_csv("dataset_humano_respuestas_completo.csv",index=True)
dataset_animal_respuestas_completo.to_csv("dataset_animal_respuestas_completo.csv",index=True)
dataset_objeto_respuestas_completo.to_csv("dataset_objeto_respuestas_completo.csv",index=True)
dataset_paisaje_respuestas_completo.to_csv("dataset_paisaje_respuestas_completo.csv",index=True)

## voy a guardar los unicos
unicos_humanos.to_csv("unicos_humanos.csv",index=True)
unicos_animal.to_csv("unicos_animal.csv",index=True)
unicos_objetos.to_csv("unicos_objetos.csv",index=True)
unicos_paisajes.to_csv("unicos_paisajes.csv",index=True)


In [6]:
unicos_humanos=pd.read_csv("unicos_humanos.csv",usecols=lambda columna: columna != 'Unnamed: 0')
unicos_animal=pd.read_csv("unicos_animal.csv",usecols=lambda columna: columna != 'Unnamed: 0')
unicos_objetos=pd.read_csv("unicos_objetos.csv",usecols=lambda columna: columna != 'Unnamed: 0')
unicos_paisajes=pd.read_csv("unicos_paisajes.csv",usecols=lambda columna: columna != 'Unnamed: 0')

dataset_humano_respuestas_completo=pd.read_csv("dataset_humano_respuestas_completo.csv",index_col='indice_2')
dataset_animal_respuestas_completo=pd.read_csv("dataset_animal_respuestas_completo.csv",index_col='indice_2')
dataset_objeto_respuestas_completo=pd.read_csv("dataset_objeto_respuestas_completo.csv",index_col='indice_2')
dataset_paisaje_respuestas_completo=pd.read_csv("dataset_paisaje_respuestas_completo.csv",index_col='indice_2')

# TODO metricas de semantica (modelos), metrica de imagenes

# Filtro 2 (Counterfractuals)

# Humano levestehin

In [115]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_humanos)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_humano_respuestas_completo[(dataset_humano_respuestas_completo['etiqueta']==unicos_humanos['etiqueta'][i]) &
                                   (dataset_humano_respuestas_completo['singularidad']==unicos_humanos['singularidad'][i])])

In [116]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_levestehin(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # Enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)


0it [00:00, ?it/s]

0 58


1it [00:00,  4.47it/s]

1 188


2it [00:00,  4.50it/s]

...

57it [00:11,  4.84it/s]

56 5853
57 5921


59it [00:12,  4.94it/s]

58 6223
59 6313


60it [00:12,  4.81it/s]


In [117]:
df_resultado_humano_levestein=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_humano_levestein.to_csv('df_resultado_humano_levestein_filtro_2_3.csv', index=False) # modificar

# Animal levestehin

In [118]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_animal)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_animal_respuestas_completo[(dataset_animal_respuestas_completo['etiqueta']==unicos_animal['etiqueta'][i]) &
                                   (dataset_animal_respuestas_completo['singularidad']==unicos_animal['singularidad'][i])])

In [119]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_levestehin(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

3it [00:00, 24.57it/s]

0 0
1 137
2 180
3 224
4 278
5 351


9it [00:00, 27.34it/s]

6 430
7 529
8 683
9 686
10 762
11 818


13it [00:00, 26.85it/s]


...

0it [00:00, ?it/s]

0 313
1 372
2 449


3it [00:00, 23.16it/s]

3 613
4 869


5it [00:00, 23.37it/s]


In [120]:
df_resultado_animal_levestein=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_animal_levestein.to_csv('df_resultado_animal_levestein_filtro_2_3.csv', index=False) # modificar

# Objeto levenstehin

In [121]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_objetos)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_objeto_respuestas_completo[(dataset_objeto_respuestas_completo['etiqueta']==unicos_objetos['etiqueta'][i]) &
                                   (dataset_objeto_respuestas_completo['singularidad']==unicos_objetos['singularidad'][i])])

In [122]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_levestehin(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

3it [00:00, 21.53it/s]

0 2
1 176
2 304
3 317
4 353


9it [00:00, 22.52it/s]

5 441
6 459
7 587
8 658
9 789


12it [00:00, 22.33it/s]


...

15 1213


3it [00:00, 29.19it/s]

0 378
1 404
2 451
3 465
4 806
5 963


8it [00:00, 27.56it/s]

6 1179
7 1211


In [123]:
df_resultado_objeto_levestein=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_objeto_levestein.to_csv('df_resultado_objeto_levestein_filtro_2_3.csv', index=False) # modificar

# Paisaje levestehin

In [124]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_paisajes)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_paisaje_respuestas_completo[(dataset_paisaje_respuestas_completo['etiqueta']==unicos_paisajes['etiqueta'][i]) &
                                   (dataset_paisaje_respuestas_completo['singularidad']==unicos_paisajes['singularidad'][i])])

In [125]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_levestehin(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

1it [00:00,  7.23it/s]

0 12
1 164


3it [00:00,  6.88it/s]

2 274
3 314


5it [00:00,  7.24it/s]

...

32 2883
33 3126


35it [00:04,  8.33it/s]

34 3292
35 3390


37it [00:04,  8.19it/s]

36 3510


In [126]:
df_resultado_paisaje_levestein=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_paisaje_levestein.to_csv('df_resultado_paisaje_levestein_filtro_2_3.csv', index=False) # modificar

# Humano overlap

In [127]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_humanos)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_humano_respuestas_completo[(dataset_humano_respuestas_completo['etiqueta']==unicos_humanos['etiqueta'][i]) &
                                   (dataset_humano_respuestas_completo['singularidad']==unicos_humanos['singularidad'][i])])

In [128]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_overlap(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # Enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

0 58


1it [00:00,  2.45it/s]

1 188


2it [00:00,  2.42it/s]

...

58it [00:24,  2.30it/s]

58 6223


59it [00:24,  2.28it/s]

59 6313


60it [00:25,  2.37it/s]


In [129]:
df_resultado_humano_overlap=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_humano_overlap.to_csv('df_resultado_humano_overlap_filtro_2_3.csv', index=False) # modificar

# Animal Overlap

In [130]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_animal)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_animal_respuestas_completo[(dataset_animal_respuestas_completo['etiqueta']==unicos_animal['etiqueta'][i]) &
                                   (dataset_animal_respuestas_completo['singularidad']==unicos_animal['singularidad'][i])])

In [131]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_overlap(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

2it [00:00, 13.41it/s]

0 0
1 137
2 180


6it [00:00, 13.88it/s]

3 224
4 278
5 351
6 430


10it [00:00, 14.61it/s]

...

2it [00:00, 14.60it/s]

2 449


4it [00:00, 14.61it/s]

3 613
4 869


5it [00:00, 14.58it/s]


In [132]:
df_resultado_animal_overlap=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_animal_overlap.to_csv('df_resultado_animal_overlap_filtro_2_3.csv', index=False) # modificar

# QUEDO AQUI

# Objeto Overlap

In [133]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_objetos)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_objeto_respuestas_completo[(dataset_objeto_respuestas_completo['etiqueta']==unicos_objetos['etiqueta'][i]) &
                                   (dataset_objeto_respuestas_completo['singularidad']==unicos_objetos['singularidad'][i])])

In [134]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_overlap(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

2it [00:00, 13.58it/s]

0 2
1 176
2 304


4it [00:00, 12.80it/s]

3 317
4 353
5 441


8it [00:00, 13.04it/s]

...

0 378
1 404
2 451


4it [00:00, 13.56it/s]

3 465
4 806
5 963


8it [00:00, 12.92it/s]

6 1179
7 1211


In [135]:
df_resultado_objeto_overlap=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_objeto_overlap.to_csv('df_resultado_objeto_overlap_filtro_2_3.csv', index=False) # modificar

# Paisaje Overlap

In [136]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_paisajes)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_paisaje_respuestas_completo[(dataset_paisaje_respuestas_completo['etiqueta']==unicos_paisajes['etiqueta'][i]) &
                                   (dataset_paisaje_respuestas_completo['singularidad']==unicos_paisajes['singularidad'][i])])

In [137]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_overlap(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

0 12


1it [00:00,  3.24it/s]

1 164


2it [00:00,  3.36it/s]

...

35it [00:08,  4.18it/s]

35 3390


36it [00:08,  4.14it/s]

36 3510


37it [00:08,  4.18it/s]


In [138]:
df_resultado_paisaje_overlap=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_paisaje_overlap.to_csv('df_resultado_paisaje_overlap_filtro_2_3.csv', index=False) # modificar

# Humano Cosine

In [139]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_humanos)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_humano_respuestas_completo[(dataset_humano_respuestas_completo['etiqueta']==unicos_humanos['etiqueta'][i]) &
                                   (dataset_humano_respuestas_completo['singularidad']==unicos_humanos['singularidad'][i])])

In [140]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_cosine(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # Enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

0 58


1it [00:00,  2.51it/s]

1 188


2it [00:00,  2.51it/s]

...

58it [00:24,  2.39it/s]

58 6223


59it [00:25,  2.35it/s]

59 6313


60it [00:25,  2.35it/s]


In [141]:
df_resultado_humano_cosine=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_humano_cosine.to_csv('df_resultado_humano_cosine_filtro_2_3.csv', index=False) # modificar

# Animal Cosine

In [142]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_animal)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_animal_respuestas_completo[(dataset_animal_respuestas_completo['etiqueta']==unicos_animal['etiqueta'][i]) &
                                   (dataset_animal_respuestas_completo['singularidad']==unicos_animal['singularidad'][i])])

In [143]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_cosine(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

2it [00:00, 13.50it/s]

0 0
1 137
2 180


4it [00:00, 13.73it/s]

3 224
4 278
5 351


8it [00:00, 13.69it/s]

...

2it [00:00, 14.03it/s]

2 449
3 613


4it [00:00, 14.30it/s]

4 869


5it [00:00, 14.26it/s]


In [144]:
df_resultado_animal_cosine=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_animal_cosine.to_csv('df_resultado_animal_cosine_filtro_2_3.csv', index=False) # modificar

# Objeto cosine

In [145]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_objetos)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_objeto_respuestas_completo[(dataset_objeto_respuestas_completo['etiqueta']==unicos_objetos['etiqueta'][i]) &
                                   (dataset_objeto_respuestas_completo['singularidad']==unicos_objetos['singularidad'][i])])

In [146]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_cosine(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

2it [00:00, 11.80it/s]

0 2
1 176
2 304


4it [00:00, 12.19it/s]

3 317
4 353
5 441


8it [00:00, 12.76it/s]

...

0 378
1 404
2 451


4it [00:00, 14.18it/s]

3 465
4 806
5 963


8it [00:00, 13.68it/s]

6 1179
7 1211


In [147]:
df_resultado_objeto_cosine=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_objeto_cosine.to_csv('df_resultado_objeto_cosine_filtro_2_3.csv', index=False) # modificar

# Paisaje cosine

In [148]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_paisajes)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_paisaje_respuestas_completo[(dataset_paisaje_respuestas_completo['etiqueta']==unicos_paisajes['etiqueta'][i]) &
                                   (dataset_paisaje_respuestas_completo['singularidad']==unicos_paisajes['singularidad'][i])])

In [149]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_cosine(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

0 12


1it [00:00,  3.42it/s]

1 164


2it [00:00,  3.69it/s]

...

35it [00:08,  4.18it/s]

35 3390


36it [00:08,  4.20it/s]

36 3510


37it [00:08,  4.20it/s]


In [150]:
df_resultado_paisaje_cosine=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_paisaje_cosine.to_csv('df_resultado_paisaje_cosine_filtro_2_3.csv', index=False) # modificar

# Humano jaccard

In [151]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_humanos)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_humano_respuestas_completo[(dataset_humano_respuestas_completo['etiqueta']==unicos_humanos['etiqueta'][i]) &
                                   (dataset_humano_respuestas_completo['singularidad']==unicos_humanos['singularidad'][i])])

In [152]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_jaccard(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # Enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

0 58


1it [00:00,  2.23it/s]

1 188


2it [00:00,  2.17it/s]

...

58it [00:24,  2.34it/s]

58 6223


59it [00:25,  2.33it/s]

59 6313


60it [00:25,  2.34it/s]


In [153]:
df_resultado_humano_jaccard=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_humano_jaccard.to_csv('df_resultado_humano_jaccard_filtro_2_3.csv', index=False) # modificar

# Animal Jaccard

In [154]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_animal)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_animal_respuestas_completo[(dataset_animal_respuestas_completo['etiqueta']==unicos_animal['etiqueta'][i]) &
                                   (dataset_animal_respuestas_completo['singularidad']==unicos_animal['singularidad'][i])])

In [155]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_jaccard(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

2it [00:00, 13.28it/s]

0 0
1 137
2 180


4it [00:00, 13.39it/s]

3 224
4 278
5 351


8it [00:00, 13.59it/s]

...

2it [00:00, 13.85it/s]

2 449


4it [00:00, 13.78it/s]

3 613
4 869


5it [00:00, 13.74it/s]


In [156]:
df_resultado_animal_jaccard=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_animal_jaccard.to_csv('df_resultado_animal_jaccard_filtro_2_3.csv', index=False) # modificar

# Objeto jaccard

In [157]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_objetos)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_objeto_respuestas_completo[(dataset_objeto_respuestas_completo['etiqueta']==unicos_objetos['etiqueta'][i]) &
                                   (dataset_objeto_respuestas_completo['singularidad']==unicos_objetos['singularidad'][i])])

In [158]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_jaccard(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

2it [00:00, 12.79it/s]

0 2
1 176
2 304


4it [00:00, 12.93it/s]

3 317
4 353
5 441


8it [00:00, 13.36it/s]

...

4it [00:00, 14.02it/s]

3 465
4 806
5 963


6it [00:00, 14.03it/s]

6 1179
7 1211


8it [00:00, 13.90it/s]


In [159]:
df_resultado_objeto_jaccard=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_objeto_jaccard.to_csv('df_resultado_objeto_jaccard_filtro_2_3.csv', index=False) # modificar

# Paisaje Jaccard

In [160]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_paisajes)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_paisaje_respuestas_completo[(dataset_paisaje_respuestas_completo['etiqueta']==unicos_paisajes['etiqueta'][i]) &
                                   (dataset_paisaje_respuestas_completo['singularidad']==unicos_paisajes['singularidad'][i])])

In [161]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_jaccard(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

0 12


1it [00:00,  3.66it/s]

1 164


2it [00:00,  3.72it/s]

...

35it [00:09,  3.85it/s]

35 3390


36it [00:09,  3.85it/s]

36 3510


37it [00:09,  3.88it/s]


In [162]:
df_resultado_paisaje_jaccard=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_paisaje_jaccard.to_csv('df_resultado_paisaje_jaccard_filtro_2_3.csv', index=False) # modificar

# Humano Bag

In [163]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_humanos)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_humano_respuestas_completo[(dataset_humano_respuestas_completo['etiqueta']==unicos_humanos['etiqueta'][i]) &
                                   (dataset_humano_respuestas_completo['singularidad']==unicos_humanos['singularidad'][i])])

In [164]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_bag(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # Enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

0 58


1it [00:00,  2.14it/s]

1 188


2it [00:00,  2.13it/s]

...

58it [00:27,  2.16it/s]

58 6223


59it [00:27,  2.14it/s]

59 6313


60it [00:28,  2.14it/s]


In [165]:
df_resultado_humano_bag=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_humano_bag.to_csv('df_resultado_humano_bag_filtro_2_3.csv', index=False) # modificar

# Animal bag

In [166]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_animal)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_animal_respuestas_completo[(dataset_animal_respuestas_completo['etiqueta']==unicos_animal['etiqueta'][i]) &
                                   (dataset_animal_respuestas_completo['singularidad']==unicos_animal['singularidad'][i])])

In [167]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_bag(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

2it [00:00, 12.85it/s]

0 0
1 137
2 180


4it [00:00, 12.99it/s]

3 224
4 278
5 351


8it [00:00, 13.30it/s]

...

2it [00:00, 12.75it/s]

2 449


4it [00:00, 12.76it/s]

3 613
4 869


5it [00:00, 12.69it/s]


In [168]:
df_resultado_animal_bag=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_animal_bag.to_csv('df_resultado_animal_bag_filtro_2_3.csv', index=False) # modificar

# Objeto Bag

In [169]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_objetos)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_objeto_respuestas_completo[(dataset_objeto_respuestas_completo['etiqueta']==unicos_objetos['etiqueta'][i]) &
                                   (dataset_objeto_respuestas_completo['singularidad']==unicos_objetos['singularidad'][i])])

In [170]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_bag(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

2it [00:00, 11.97it/s]

0 2
1 176
2 304


4it [00:00, 11.97it/s]

3 317
4 353
5 441


8it [00:00, 12.00it/s]

...

4it [00:00, 13.18it/s]

3 465
4 806
5 963


6it [00:00, 13.09it/s]

6 1179
7 1211


8it [00:00, 13.07it/s]


In [171]:
df_resultado_objeto_bag=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_objeto_bag.to_csv('df_resultado_objeto_bag_filtro_2_3.csv', index=False) # modificar

# Paisaje Bag

In [172]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_paisajes)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_paisaje_respuestas_completo[(dataset_paisaje_respuestas_completo['etiqueta']==unicos_paisajes['etiqueta'][i]) &
                                   (dataset_paisaje_respuestas_completo['singularidad']==unicos_paisajes['singularidad'][i])])

In [173]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_bag(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

0 12


1it [00:00,  3.22it/s]

1 164


2it [00:00,  3.36it/s]

...

35it [00:09,  3.60it/s]

35 3390


36it [00:09,  3.60it/s]

36 3510


37it [00:10,  3.61it/s]


In [174]:
df_resultado_paisaje_bag=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_paisaje_bag.to_csv('df_resultado_paisaje_bag_filtro_2_3.csv', index=False) # modificar

# Empezamos filtro 1 metrica de semantica similarityweb

# Caso Humanos

In [7]:
subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_humanos)):
    #AQUI TAMBIEN AÑADIR LOS OTROS DATASETS
    subdataset_para_comparativa_filtro_1.append(dataset_humano_respuestas_completo[(dataset_humano_respuestas_completo['etiqueta']==unicos_humanos['etiqueta'][i]) &
                                   (dataset_humano_respuestas_completo['singularidad']==unicos_humanos['singularidad'][i])])

In [8]:
# Suponiendo que similitud_metrica_levestehin y subdataset_para_comparativa_filtro_1 están definidos

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_websimilarity(texto_query, texto_iteracion)
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)

62it [55:21, 53.57s/it]
67it [1:04:32, 57.80s/it]
61it [53:32, 52.66s/it]
60it [51:55, 51.93s/it]


In [9]:
df_resultado_humano_similarity=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_humano_similarity.to_csv('df_resultado_humano_similarity_3.csv', index=False) # modificar

# Caso animal

In [10]:
subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_animal)):
    
    subdataset_para_comparativa_filtro_1.append(dataset_animal_respuestas_completo[(dataset_animal_respuestas_completo['etiqueta']==unicos_animal['etiqueta'][i]) &
                                   (dataset_animal_respuestas_completo['singularidad']==unicos_animal['singularidad'][i])])

In [11]:
# Suponiendo que similitud_metrica_levestehin y subdataset_para_comparativa_filtro_1 están definidos

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_websimilarity(texto_query, texto_iteracion)
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)

13it [02:23, 11.01s/it]
10it [01:17,  7.80s/it]
7it [00:36,  5.19s/it]
10it [01:17,  7.79s/it]
12it [01:53,  9.50s/it]
8it [00:48,  6.05s/it]
14it [02:37, 11.24s/it]
10it [01:18,  7.80s/it]
15it [03:00, 12.06s/it]
8it [00:48,  6.08s/it]
11it [01:34,  8.63s/it]
9it [01:02,  6.95s/it]
9it [01:02,  6.94s/it]
9it [01:02,  6.95s/it]
7it [00:36,  5.21s/it]
8it [00:48,  6.06s/it]
17it [03:54, 13.81s/it]
8it [00:48,  6.08s/it]
4it [00:10,  2.62s/it]
6it [00:26,  4.37s/it]
10it [01:18,  7.81s/it]
6it [00:26,  4.37s/it]
7it [00:36,  5.19s/it]
11it [01:35,  8.68s/it]
10it [01:18,  7.83s/it]
6it [00:26,  4.39s/it]
5it [00:17,  3.49s/it]


In [12]:
df_resultado_animal_similarity=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_animal_similarity.to_csv('df_resultado_animal_similarity_3.csv', index=False) # modificar

# Caso Objeto

In [13]:
# para trabajar con los cercanos dividimos en sub dataset por etiqueta
subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_objetos)):
    # modificar dataset
    subdataset_para_comparativa_filtro_1.append(dataset_objeto_respuestas_completo[(dataset_objeto_respuestas_completo['etiqueta']==unicos_objetos['etiqueta'][i]) &
                                   (dataset_objeto_respuestas_completo['singularidad']==unicos_objetos['singularidad'][i])])

In [14]:
# Suponiendo que similitud_metrica_levestehin y subdataset_para_comparativa_filtro_1 están definidos

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_websimilarity(texto_query, texto_iteracion)
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)

12it [01:44,  8.75s/it]
9it [00:56,  6.30s/it]
11it [01:26,  7.82s/it]
14it [02:22, 10.20s/it]
15it [02:44, 10.96s/it]
18it [03:59, 13.31s/it]
9it [00:56,  6.29s/it]
12it [01:44,  8.68s/it]
16it [03:08, 11.77s/it]
12it [01:43,  8.63s/it]
14it [02:23, 10.24s/it]
10it [01:11,  7.13s/it]
10it [01:10,  7.08s/it]
16it [03:07, 11.74s/it]
12it [01:43,  8.65s/it]
12it [01:42,  8.58s/it]
10it [01:11,  7.10s/it]
5it [00:15,  3.16s/it]
9it [00:56,  6.30s/it]
16it [03:07, 11.73s/it]
8it [00:43,  5.50s/it]


In [15]:
df_resultado_objeto_similarity=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_objeto_similarity.to_csv('df_resultado_objeto_similarity_3.csv', index=False) # modificar

# Caso paisaje

In [16]:
subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_paisajes)):
    
    subdataset_para_comparativa_filtro_1.append(dataset_paisaje_respuestas_completo[(dataset_paisaje_respuestas_completo['etiqueta']==unicos_paisajes['etiqueta'][i]) &
                                   (dataset_paisaje_respuestas_completo['singularidad']==unicos_paisajes['singularidad'][i])])

In [17]:
# Suponiendo que similitud_metrica_levestehin y subdataset_para_comparativa_filtro_1 están definidos

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_websimilarity(texto_query, texto_iteracion)
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)

35it [14:56, 25.63s/it]
52it [33:15, 38.37s/it]
39it [18:39, 28.70s/it]
39it [18:43, 28.80s/it]
48it [28:25, 35.54s/it]
37it [16:46, 27.21s/it]


In [18]:
df_resultado_paisaje_similarity=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_paisaje_similarity.to_csv('df_resultado_paisaje_similarity_3.csv', index=False) # modificar

# Ahora websimiliraty con filtro 2

# Caso humano

In [19]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_humanos)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_humano_respuestas_completo[(dataset_humano_respuestas_completo['etiqueta']==unicos_humanos['etiqueta'][i]) &
                                   (dataset_humano_respuestas_completo['singularidad']==unicos_humanos['singularidad'][i])])

In [20]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_websimilarity(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # Enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

0 58


1it [02:46, 166.11s/it]

1 188


2it [05:30, 165.12s/it]

...

58it [2:42:38, 168.25s/it]

58 6223


59it [2:45:27, 168.39s/it]

59 6313


60it [2:48:15, 168.26s/it]


In [21]:
df_resultado_humano_similarity_filtro2=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_humano_similarity_filtro2.to_csv('df_resultado_humano_similarity_filtro2_3.csv', index=False) # modificar

# Caso animal

In [22]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_animal)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_animal_respuestas_completo[(dataset_animal_respuestas_completo['etiqueta']==unicos_animal['etiqueta'][i]) &
                                   (dataset_animal_respuestas_completo['singularidad']==unicos_animal['singularidad'][i])])

In [23]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_websimilarity(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # Enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

0 0


1it [00:25, 25.88s/it]

1 137


2it [00:50, 25.03s/it]

...

3it [01:17, 25.79s/it]

3 613


4it [01:43, 25.78s/it]

4 869


5it [02:08, 25.80s/it]


In [24]:
df_resultado_animal_similarity_filtro2=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_animal_similarity_filtro2.to_csv('df_resultado_animal_similarity_filtro2_3.csv', index=False) # modificar

# Caso objeto

In [25]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_objetos)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_objeto_respuestas_completo[(dataset_objeto_respuestas_completo['etiqueta']==unicos_objetos['etiqueta'][i]) &
                                   (dataset_objeto_respuestas_completo['singularidad']==unicos_objetos['singularidad'][i])])

In [26]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_websimilarity(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # Enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

0 2


1it [00:28, 28.00s/it]

1 176


2it [00:54, 27.18s/it]

...

6it [02:31, 25.24s/it]

6 1179


7it [02:56, 25.27s/it]

7 1211


8it [03:21, 25.24s/it]


In [27]:
df_resultado_objeto_similarity_filtro2=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_objeto_similarity_filtro2.to_csv('df_resultado_objeto_similarity_filtro2_3.csv', index=False) # modificar

# Caso Paisaje

In [28]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_paisajes)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_paisaje_respuestas_completo[(dataset_paisaje_respuestas_completo['etiqueta']==unicos_paisajes['etiqueta'][i]) &
                                   (dataset_paisaje_respuestas_completo['singularidad']==unicos_paisajes['singularidad'][i])])

In [29]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_websimilarity(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # Enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

0 12


1it [01:39, 99.25s/it]

1 164


2it [03:16, 98.36s/it]

...

35it [56:05, 96.12s/it]

35 3390


36it [57:42, 96.26s/it]

36 3510


37it [59:18, 96.19s/it]


In [30]:
df_resultado_paisaje_similarity_filtro2=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_paisaje_similarity_filtro2.to_csv('df_resultado_paisaje_similarity_filtro2_3.csv', index=False) # modificar

# # Empezamos filtro 1 metrica de semantica Sbert

# Caso Humano

In [32]:
%pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 2.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 1.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 1.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.0/763.0 kB 2.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 2.0 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [33]:
import transformers
print(transformers.__version__)

/home/angel/miniconda3/envs/TFM_2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.49.0


In [34]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

In [35]:
# funcion metrica de similitud_2 semantica
def similitud_metrica_sbert(texto_query,texto_iteracion):
    texto_query=str(texto_query)
    texto_iteracion=str(texto_iteracion)
    if texto_query!='0':
        texto_query=model.encode([texto_query])
        texto_iteracion=model.encode([texto_iteracion])
        similitud=model.similarity(texto_query,texto_iteracion)
        return float(similitud)
    else:
        return 6666

In [36]:
subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_humanos)):
    #AQUI TAMBIEN AÑADIR LOS OTROS DATASETS
    subdataset_para_comparativa_filtro_1.append(dataset_humano_respuestas_completo[(dataset_humano_respuestas_completo['etiqueta']==unicos_humanos['etiqueta'][i]) &
                                   (dataset_humano_respuestas_completo['singularidad']==unicos_humanos['singularidad'][i])])

In [37]:
# Suponiendo que similitud_metrica_levestehin y subdataset_para_comparativa_filtro_1 están definidos

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_sbert(texto_query, texto_iteracion)
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)

62it [41:12, 39.88s/it]
67it [47:09, 42.23s/it]
61it [39:03, 38.41s/it]
60it [37:50, 37.85s/it]


In [38]:
df_resultado_humano_sbert=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_humano_sbert.to_csv('df_resultado_humano_sbert_2_3.csv', index=False) # modificar

# Caso Animal

In [39]:

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_animal)):

    subdataset_para_comparativa_filtro_1.append(dataset_animal_respuestas_completo[(dataset_animal_respuestas_completo['etiqueta']==unicos_animal['etiqueta'][i]) &
                                   (dataset_animal_respuestas_completo['singularidad']==unicos_animal['singularidad'][i])])

In [40]:

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_sbert(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # eecoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)

13it [01:46,  8.17s/it]
10it [00:57,  5.74s/it]
7it [00:26,  3.81s/it]
10it [00:57,  5.73s/it]
12it [01:24,  7.01s/it]
8it [00:35,  4.47s/it]
14it [01:55,  8.25s/it]
10it [00:57,  5.76s/it]
15it [02:13,  8.88s/it]
8it [00:35,  4.47s/it]
11it [01:10,  6.37s/it]
9it [00:45,  5.11s/it]
9it [00:45,  5.09s/it]
9it [00:46,  5.12s/it]
7it [00:26,  3.84s/it]
8it [00:35,  4.45s/it]
17it [02:52, 10.18s/it]
8it [00:35,  4.46s/it]
4it [00:07,  1.94s/it]
6it [00:19,  3.21s/it]
10it [00:57,  5.72s/it]
6it [00:19,  3.21s/it]
7it [00:26,  3.82s/it]
11it [01:10,  6.39s/it]
10it [00:57,  5.70s/it]
6it [00:19,  3.20s/it]
5it [00:12,  2.55s/it]


In [41]:
df_resultado_animal_sbert=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_animal_sbert.to_csv('df_resultado_animal_sbert_3.csv', index=False) # modificar

# Caso objeto

In [42]:

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_objetos)):

    subdataset_para_comparativa_filtro_1.append(dataset_objeto_respuestas_completo[(dataset_objeto_respuestas_completo['etiqueta']==unicos_objetos['etiqueta'][i]) &
                                   (dataset_objeto_respuestas_completo['singularidad']==unicos_objetos['singularidad'][i])])

In [43]:
def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_sbert(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # eecoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)

12it [01:17,  6.48s/it]
9it [00:41,  4.61s/it]
11it [01:03,  5.76s/it]
14it [01:44,  7.49s/it]
15it [02:01,  8.09s/it]
18it [02:56,  9.80s/it]
9it [00:41,  4.61s/it]
12it [01:16,  6.35s/it]
16it [02:18,  8.67s/it]
12it [01:16,  6.34s/it]
14it [01:45,  7.51s/it]
10it [00:52,  5.21s/it]
10it [00:51,  5.19s/it]
16it [02:18,  8.64s/it]
12it [01:16,  6.35s/it]
12it [01:16,  6.35s/it]
10it [00:51,  5.20s/it]
5it [00:11,  2.34s/it]
9it [00:41,  4.62s/it]
16it [02:18,  8.65s/it]
8it [00:32,  4.06s/it]


In [44]:
df_resultado_objeto_sbert=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_objeto_sbert.to_csv('df_resultado_objeto_sbert_3.csv', index=False) # modificar

# Caso paisaje

In [45]:

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_paisajes)):

    subdataset_para_comparativa_filtro_1.append(dataset_paisaje_respuestas_completo[(dataset_paisaje_respuestas_completo['etiqueta']==unicos_paisajes['etiqueta'][i]) &
                                   (dataset_paisaje_respuestas_completo['singularidad']==unicos_paisajes['singularidad'][i])])

In [46]:
def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_sbert(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # eecoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)

35it [10:56, 18.75s/it]
52it [24:16, 28.02s/it]
39it [13:36, 20.93s/it]
39it [13:36, 20.94s/it]
48it [20:45, 25.94s/it]
37it [12:15, 19.89s/it]


In [47]:
df_resultado_paisaje_sbert=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_paisaje_sbert.to_csv('df_resultado_paisaje_sbert_3.csv', index=False) # modificar

# Empezamos filtro 2 metrica de semantica Sbert

# Caso humano

In [48]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_humanos)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_humano_respuestas_completo[(dataset_humano_respuestas_completo['etiqueta']==unicos_humanos['etiqueta'][i]) &
                                   (dataset_humano_respuestas_completo['singularidad']==unicos_humanos['singularidad'][i])])

In [49]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_sbert(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # Enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

0 58


1it [02:02, 122.86s/it]

1 188


2it [04:03, 121.72s/it]

...

58it [2:01:00, 124.21s/it]

58 6223


59it [2:03:04, 124.18s/it]

59 6313


60it [2:05:08, 125.15s/it]


In [50]:
df_resultado_humano_sbert_filtro2=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_humano_sbert_filtro2.to_csv('df_resultado_humano_sbert_filtro2_3.csv', index=False) # modificar

# Caso animal

In [51]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_animal)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_animal_respuestas_completo[(dataset_animal_respuestas_completo['etiqueta']==unicos_animal['etiqueta'][i]) &
                                   (dataset_animal_respuestas_completo['singularidad']==unicos_animal['singularidad'][i])])

In [52]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_sbert(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # Enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

0 0


1it [00:20, 20.24s/it]

1 137


2it [00:38, 19.33s/it]

...

3it [00:58, 19.64s/it]

3 613


4it [01:18, 19.59s/it]

4 869


5it [01:37, 19.58s/it]


In [53]:
df_resultado_animal_sbert_filtro2=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_animal_sbert_filtro2.to_csv('df_resultado_animal_sbert_filtro2_3.csv', index=False) # modificar

# Caso objeto

In [54]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_objetos)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_objeto_respuestas_completo[(dataset_objeto_respuestas_completo['etiqueta']==unicos_objetos['etiqueta'][i]) &
                                   (dataset_objeto_respuestas_completo['singularidad']==unicos_objetos['singularidad'][i])])

In [55]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_sbert(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # Enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

0 2


1it [00:22, 22.04s/it]

1 176


2it [00:42, 20.88s/it]

...

6it [01:53, 18.87s/it]

6 1179


7it [02:12, 18.87s/it]

7 1211


8it [02:30, 18.86s/it]


In [56]:
df_resultado_objeto_sbert_filtro2=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_objeto_sbert_filtro2.to_csv('df_resultado_objeto_sbert_filtro2_3.csv', index=False) # modificar

# Caso Paisaje

In [57]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_paisajes)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_paisaje_respuestas_completo[(dataset_paisaje_respuestas_completo['etiqueta']==unicos_paisajes['etiqueta'][i]) &
                                   (dataset_paisaje_respuestas_completo['singularidad']==unicos_paisajes['singularidad'][i])])

In [58]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_sbert(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # Enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

0 12


1it [01:15, 75.80s/it]

1 164


2it [02:29, 74.36s/it]

...

35it [41:34, 71.34s/it]

35 3390


36it [42:45, 71.28s/it]

36 3510


37it [43:56, 71.26s/it]


In [59]:
df_resultado_paisaje_sbert_filtro2=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_paisaje_sbert_filtro2.to_csv('df_resultado_paisaje_sbert_filtro2_3.csv', index=False) # modificar

# A partir de aqui seria el note 9 de metricas processiong, esto es investigacion:: ACtualizacion creo que no porque sigue la metrica con pixel to pixel

In [7]:
unicos_humanos=pd.read_csv("unicos_humanos.csv",usecols=lambda columna: columna != 'Unnamed: 0')
unicos_animal=pd.read_csv("unicos_animal.csv",usecols=lambda columna: columna != 'Unnamed: 0')
unicos_objetos=pd.read_csv("unicos_objetos.csv",usecols=lambda columna: columna != 'Unnamed: 0')
unicos_paisajes=pd.read_csv("unicos_paisajes.csv",usecols=lambda columna: columna != 'Unnamed: 0')

dataset_humano_respuestas_completo=pd.read_csv("dataset_humano_respuestas_completo.csv",index_col='indice_2')
dataset_animal_respuestas_completo=pd.read_csv("dataset_animal_respuestas_completo.csv",index_col='indice_2')
dataset_objeto_respuestas_completo=pd.read_csv("dataset_objeto_respuestas_completo.csv",index_col='indice_2')
dataset_paisaje_respuestas_completo=pd.read_csv("dataset_paisaje_respuestas_completo.csv",index_col='indice_2')

In [8]:
import json

def preservar_formato(x):
    return json.dumps(x.tolist())

dataset_humano_cifar['indice_2']=dataset_humano_cifar.index
dataset_humano_cifar['data_lista']=dataset_humano_cifar[b'data'].apply(preservar_formato)

dataset_animal_cifar['indice_2']=dataset_animal_cifar.index
dataset_animal_cifar['data_lista']=dataset_animal_cifar[b'data'].apply(preservar_formato)

dataset_objeto_cifar['indice_2']=dataset_objeto_cifar.index
dataset_objeto_cifar['data_lista']=dataset_objeto_cifar[b'data'].apply(preservar_formato)

dataset_paisaje_cifar['indice_2']=dataset_paisaje_cifar.index
dataset_paisaje_cifar['data_lista']=dataset_paisaje_cifar[b'data'].apply(preservar_formato)


# TODO metricas de pixeltopixel

### La columna b'data' de los dataset ...respuestas completo parece estar rota, se recupera la info original de los bytes de la imagenes desde los dataset_cifar.... para ello la siguiente consulta

In [11]:
query_humano_recovery_image_bytes='''
SELECT dataset_humano_respuestas_completo.*, dataset_humano_cifar.data_lista FROM dataset_humano_respuestas_completo, dataset_humano_cifar
WHERE dataset_humano_respuestas_completo.indice_2=dataset_humano_cifar.indice_2
'''

query_animal_recovery_image_bytes='''
SELECT dataset_animal_respuestas_completo.*, dataset_animal_cifar.data_lista FROM dataset_animal_respuestas_completo, dataset_animal_cifar
WHERE dataset_animal_respuestas_completo.indice_2=dataset_animal_cifar.indice_2
'''

query_objeto_recovery_image_bytes='''
SELECT dataset_objeto_respuestas_completo.*, dataset_objeto_cifar.data_lista FROM dataset_objeto_respuestas_completo, dataset_objeto_cifar
WHERE dataset_objeto_respuestas_completo.indice_2=dataset_objeto_cifar.indice_2
'''

query_paisaje_recovery_image_bytes='''
SELECT dataset_paisaje_respuestas_completo.*, dataset_paisaje_cifar.data_lista FROM dataset_paisaje_respuestas_completo, dataset_paisaje_cifar
WHERE dataset_paisaje_respuestas_completo.indice_2=dataset_paisaje_cifar.indice_2
'''

dataset_humano_respuestas_completo_bytes_reparado=sql_query(query_humano_recovery_image_bytes)

dataset_animal_respuestas_completo_bytes_reparado=sql_query(query_animal_recovery_image_bytes)

dataset_objeto_respuestas_completo_bytes_reparado=sql_query(query_objeto_recovery_image_bytes)

dataset_paisaje_respuestas_completo_bytes_reparado=sql_query(query_paisaje_recovery_image_bytes)


In [14]:
def recuperar_data_array(x): # esto es necesario para que el string que contiene la imagen se interprete de forma correcta como una estructura de datos y poder acceder a los bytes
    return np.array(json.loads(x),dtype=np.uint8) 

dataset_humano_respuestas_completo_bytes_reparado['data_lista']= dataset_humano_respuestas_completo_bytes_reparado['data_lista'].apply(recuperar_data_array)


dataset_animal_respuestas_completo_bytes_reparado['data_lista']= dataset_animal_respuestas_completo_bytes_reparado['data_lista'].apply(recuperar_data_array)


dataset_objeto_respuestas_completo_bytes_reparado['data_lista']= dataset_objeto_respuestas_completo_bytes_reparado['data_lista'].apply(recuperar_data_array)


dataset_paisaje_respuestas_completo_bytes_reparado['data_lista']= dataset_paisaje_respuestas_completo_bytes_reparado['data_lista'].apply(recuperar_data_array)


In [15]:
### resetear index
dataset_humano_respuestas_completo_bytes_reparado.set_index(dataset_humano_respuestas_completo_bytes_reparado['indice_2'],inplace=True)
dataset_humano_respuestas_completo_bytes_reparado = dataset_humano_respuestas_completo_bytes_reparado.rename(columns={'indice_2': 'indice_main'})

In [16]:
### resetear index animal
dataset_animal_respuestas_completo_bytes_reparado.set_index(dataset_animal_respuestas_completo_bytes_reparado['indice_2'],inplace=True)
dataset_animal_respuestas_completo_bytes_reparado = dataset_animal_respuestas_completo_bytes_reparado.rename(columns={'indice_2': 'indice_main'})

In [17]:
### resetear index objeto
dataset_objeto_respuestas_completo_bytes_reparado.set_index(dataset_objeto_respuestas_completo_bytes_reparado['indice_2'],inplace=True)
dataset_objeto_respuestas_completo_bytes_reparado = dataset_objeto_respuestas_completo_bytes_reparado.rename(columns={'indice_2': 'indice_main'})

In [18]:
### resetear index paisaje
dataset_paisaje_respuestas_completo_bytes_reparado.set_index(dataset_paisaje_respuestas_completo_bytes_reparado['indice_2'],inplace=True)
dataset_paisaje_respuestas_completo_bytes_reparado = dataset_paisaje_respuestas_completo_bytes_reparado.rename(columns={'indice_2': 'indice_main'})

In [19]:
from PIL import ImageChops
'''def similitud_metrica_pixel_to_pixel(image,image2):

    image=Image.fromarray(image)
    image2=Image.fromarray(image2)

    diferencia = ImageChops.difference(image, image2)
    promedio_diferencia = np.mean(np.abs(np.array(diferencia)))

    return promedio_diferencia
    # No existe el 6666 ya que no hay preguntas


CREO QUE LA FUNCION SERIA:

'''
def similitud_metrica_pixel_to_pixel(image,image2):

    image=Image.fromarray(image.reshape(3,32,32).transpose(1,2,0))
    image2=Image.fromarray(image2.reshape(3,32,32).transpose(1,2,0))

    diferencia = ImageChops.difference(image, image2)
    promedio_diferencia = np.mean(np.abs(np.array(diferencia)))

    return promedio_diferencia


pARECE QUE HACE FALTA TRANSPORNER LA IMAGEN

# Filtro 1

## No eliminar para comentar despues

In [20]:
image = dataset_humano_cifar[b'data'][58]
image = image.reshape(3, 32, 32).transpose(1, 2, 0)

image2=dataset_humano_cifar[b'data'][65]
image2= image2.reshape(3,32,32).transpose(1,2,0)

prueba=np.mean(abs(image2-image))
print(prueba)
#image=Image.fromarray(image)
#image2=Image.fromarray(image2)

#pixel_to_pixel=ImageChops.difference(image,image2)



138.83951822916666


# Caso humano

In [24]:
subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_humanos)):
    #AQUI TAMBIEN AÑADIR LOS OTROS DATASETS
    subdataset_para_comparativa_filtro_1.append(dataset_humano_respuestas_completo_bytes_reparado[(dataset_humano_respuestas_completo_bytes_reparado['etiqueta']==unicos_humanos['etiqueta'][i]) &
                                   (dataset_humano_respuestas_completo_bytes_reparado['singularidad']==unicos_humanos['singularidad'][i])])

In [25]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# Suponiendo que similitud_metrica_levestehin y subdataset_para_comparativa_filtro_1 están definidos

def comparar_imagenes(caso_query, resto_query):
    resultados = []
    imagen_query = caso_query.loc[caso_query.index.tolist()[0], 'data_lista']
    
    for x in resto_query.index.tolist():
        imagen_iteracion = resto_query.loc[x, 'data_lista']
        comparacion = similitud_metrica_pixel_to_pixel(imagen_query, imagen_iteracion)
        resultados.append((int(caso_query['indice_main'].iloc[0]), x, 'data_lista', comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # Crear una lista de futuros
            futuros = []
            #for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
            futuros.append(executor.submit(comparar_imagenes, caso_query, resto_query))
            
            # Recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)

62it [00:00, 140.81it/s]
67it [00:00, 137.67it/s]
61it [00:00, 156.00it/s]
60it [00:00, 145.03it/s]


In [26]:
df_resultado_humano_pixel_to_pixel=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_humano_pixel_to_pixel.to_csv('df_resultado_humano_pixel_to_pixel_v2_3.csv', index=False) # modificar

# Caso animal

In [27]:
subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_animal)):
    
    subdataset_para_comparativa_filtro_1.append(dataset_animal_respuestas_completo_bytes_reparado[(dataset_animal_respuestas_completo_bytes_reparado['etiqueta']==unicos_animal['etiqueta'][i]) &
                                   (dataset_animal_respuestas_completo_bytes_reparado['singularidad']==unicos_animal['singularidad'][i])])

In [28]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# Suponiendo que similitud_metrica_levestehin y subdataset_para_comparativa_filtro_1 están definidos

def comparar_imagenes(caso_query, resto_query):
    resultados = []
    imagen_query = caso_query.loc[caso_query.index.tolist()[0], 'data_lista']
    
    for x in resto_query.index.tolist():
        imagen_iteracion = resto_query.loc[x, 'data_lista']
        comparacion = similitud_metrica_pixel_to_pixel(imagen_query, imagen_iteracion)
        resultados.append((int(caso_query['indice_main'].iloc[0]), x, 'data_lista', comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # Crear una lista de futuros
            futuros = []
            #for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
            futuros.append(executor.submit(comparar_imagenes, caso_query, resto_query))
            
            # Recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)

13it [00:00, 493.81it/s]
10it [00:00, 627.61it/s]
7it [00:00, 739.03it/s]
10it [00:00, 638.31it/s]
12it [00:00, 572.61it/s]
8it [00:00, 785.95it/s]
14it [00:00, 533.26it/s]
10it [00:00, 688.18it/s]
15it [00:00, 553.66it/s]
8it [00:00, 608.85it/s]
11it [00:00, 638.26it/s]
9it [00:00, 720.89it/s]
9it [00:00, 738.88it/s]
9it [00:00, 607.54it/s]
7it [00:00, 872.72it/s]
8it [00:00, 758.58it/s]
17it [00:00, 439.04it/s]
8it [00:00, 783.29it/s]
4it [00:00, 996.92it/s]
6it [00:00, 875.24it/s]
10it [00:00, 681.03it/s]
6it [00:00, 877.65it/s]
7it [00:00, 746.53it/s]
11it [00:00, 614.67it/s]
10it [00:00, 682.82it/s]
6it [00:00, 881.19it/s]
5it [00:00, 826.79it/s]


In [29]:
df_resultado_animal_pixel_to_pixel=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_animal_pixel_to_pixel.to_csv('df_resultado_animal_pixel_to_pixel_v2_3.csv', index=False) # modificar

# Caso objeto

In [30]:
subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_objetos)):
    
    subdataset_para_comparativa_filtro_1.append(dataset_objeto_respuestas_completo_bytes_reparado[(dataset_objeto_respuestas_completo_bytes_reparado['etiqueta']==unicos_objetos['etiqueta'][i]) &
                                   (dataset_objeto_respuestas_completo_bytes_reparado['singularidad']==unicos_objetos['singularidad'][i])])

In [31]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# Suponiendo que similitud_metrica_levestehin y subdataset_para_comparativa_filtro_1 están definidos

def comparar_imagenes(caso_query, resto_query):
    resultados = []
    imagen_query = caso_query.loc[caso_query.index.tolist()[0], 'data_lista']
    
    for x in resto_query.index.tolist():
        imagen_iteracion = resto_query.loc[x, 'data_lista']
        comparacion = similitud_metrica_pixel_to_pixel(imagen_query, imagen_iteracion)
        resultados.append((int(caso_query['indice_main'].iloc[0]), x, 'data_lista', comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # Crear una lista de futuros
            futuros = []
            #for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
            futuros.append(executor.submit(comparar_imagenes, caso_query, resto_query))
            
            # Recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)

12it [00:00, 391.00it/s]
9it [00:00, 527.35it/s]
11it [00:00, 455.20it/s]
14it [00:00, 435.30it/s]
15it [00:00, 415.54it/s]
18it [00:00, 348.63it/s]
9it [00:00, 560.74it/s]
12it [00:00, 482.91it/s]
16it [00:00, 374.69it/s]
12it [00:00, 435.09it/s]
14it [00:00, 404.32it/s]
10it [00:00, 440.70it/s]
10it [00:00, 417.40it/s]
16it [00:00, 374.44it/s]
12it [00:00, 418.10it/s]
12it [00:00, 469.76it/s]
10it [00:00, 591.25it/s]
5it [00:00, 775.43it/s]
9it [00:00, 662.41it/s]
16it [00:00, 382.81it/s]
8it [00:00, 563.47it/s]


In [32]:
df_resultado_objeto_pixel_to_pixel=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_objeto_pixel_to_pixel.to_csv('df_resultado_objeto_pixel_to_pixel_v2_3.csv', index=False) # modificar

# Caso papisaje

In [33]:
subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_paisajes)):
    
    subdataset_para_comparativa_filtro_1.append(dataset_paisaje_respuestas_completo_bytes_reparado[(dataset_paisaje_respuestas_completo_bytes_reparado['etiqueta']==unicos_paisajes['etiqueta'][i]) &
                                   (dataset_paisaje_respuestas_completo_bytes_reparado['singularidad']==unicos_paisajes['singularidad'][i])])

In [34]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# Suponiendo que similitud_metrica_levestehin y subdataset_para_comparativa_filtro_1 están definidos

def comparar_imagenes(caso_query, resto_query):
    resultados = []
    imagen_query = caso_query.loc[caso_query.index.tolist()[0], 'data_lista']
    
    for x in resto_query.index.tolist():
        imagen_iteracion = resto_query.loc[x, 'data_lista']
        comparacion = similitud_metrica_pixel_to_pixel(imagen_query, imagen_iteracion)
        resultados.append((int(caso_query['indice_main'].iloc[0]), x, 'data_lista', comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # Crear una lista de futuros
            futuros = []
            #for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
            futuros.append(executor.submit(comparar_imagenes, caso_query, resto_query))
            
            # Recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)

35it [00:00, 207.44it/s]
52it [00:00, 154.29it/s]
39it [00:00, 189.30it/s]
39it [00:00, 193.45it/s]
48it [00:00, 164.62it/s]
37it [00:00, 206.67it/s]


In [35]:
df_resultado_paisaje_pixel_to_pixel=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_paisaje_pixel_to_pixel.to_csv('df_resultado_paisaje_pixel_to_pixel_v2_3.csv', index=False) # modificar

# Filtro 2 pixel to pixel

# Caso Humano

In [36]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_humanos)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_humano_respuestas_completo_bytes_reparado[(dataset_humano_respuestas_completo_bytes_reparado['etiqueta']==unicos_humanos['etiqueta'][i]) &
                                   (dataset_humano_respuestas_completo_bytes_reparado['singularidad']==unicos_humanos['singularidad'][i])])

In [37]:
# empezamos a calcular las metricas
def comparar_imagenes(caso_query, resto_query):
    resultados = []
    imagen_query = caso_query.loc[caso_query.index.tolist()[0], 'data_lista']
    
    for x in resto_query.index.tolist():
        imagen_iteracion = resto_query.loc[x, 'data_lista']
        comparacion = similitud_metrica_pixel_to_pixel(imagen_query, imagen_iteracion)
        resultados.append((int(caso_query['indice_main'].iloc[0]), x, 'data_lista', comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                #for campo in caso_query.columns[4:-1].tolist():
                    # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_imagenes, caso_query, resto_query))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

6it [00:00, 56.71it/s]

0 58
1 188
2 323
3 490
4 578
5 598
6 637
7 657
8 692
9 707
10 740
11 767


18it [00:00, 57.35it/s]

12 888
13 929
14 968
15 1021
16 1146
17 1267
18 1291
19 1311
20 1328
21 1451
22 1576
23 1772


30it [00:00, 56.51it/s]

...

45 5019
46 5164
47 5180
48 5229
49 5335
50 5390
51 5417
52 5480
53 5487


54it [00:01, 52.00it/s]

54 5661
55 5802


60it [00:01, 52.95it/s]

56 5853
57 5921
58 6223
59 6313


In [38]:
df_resultado_humano_pixel_to_pixel_filtro_2=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_humano_pixel_to_pixel_filtro_2.to_csv('df_resultado_humano_pixel_to_pixel_filtro_2_v2_3.csv', index=False) # modificar

# Caso animal

In [39]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_animal)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_animal_respuestas_completo_bytes_reparado[(dataset_animal_respuestas_completo_bytes_reparado['etiqueta']==unicos_animal['etiqueta'][i]) &
                                   (dataset_animal_respuestas_completo_bytes_reparado['singularidad']==unicos_animal['singularidad'][i])])

In [40]:
# empezamos a calcular las metricas
def comparar_imagenes(caso_query, resto_query):
    resultados = []
    imagen_query = caso_query.loc[caso_query.index.tolist()[0], 'data_lista']
    
    for x in resto_query.index.tolist():
        imagen_iteracion = resto_query.loc[x, 'data_lista']
        comparacion = similitud_metrica_pixel_to_pixel(imagen_query, imagen_iteracion)
        resultados.append((int(caso_query['indice_main'].iloc[0]), x, 'data_lista', comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                #for campo in caso_query.columns[4:-1].tolist():
                    # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_imagenes, caso_query, resto_query))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

13it [00:00, 222.92it/s]


0 0
1 137
2 180
3 224
4 278
5 351
6 430
7 529
8 683
9 686
10 762
11 818
12 870


10it [00:00, 182.41it/s]


0 1
1 155
2 238
3 343
4 453
5 677
6 711
7 793
8 906
9 914


...

4 623
5 661


0it [00:00, ?it/s]

0 313
1 372
2 449


5it [00:00, 201.92it/s]

3 613
4 869


In [41]:
df_resultado_animal_pixel_to_pixel_filtro_2=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_animal_pixel_to_pixel_filtro_2.to_csv('df_resultado_animal_pixel_to_pixel_filtro_2_v2_3.csv', index=False) # modificar

# Caso objeto

In [42]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_objetos)):
    
    subdataset_para_comparativa_filtro_2.append(dataset_objeto_respuestas_completo_bytes_reparado[(dataset_objeto_respuestas_completo_bytes_reparado['etiqueta']==unicos_objetos['etiqueta'][i]) &
                                   (dataset_objeto_respuestas_completo_bytes_reparado['singularidad']==unicos_objetos['singularidad'][i])])

In [43]:
# empezamos a calcular las metricas
def comparar_imagenes(caso_query, resto_query):
    resultados = []
    imagen_query = caso_query.loc[caso_query.index.tolist()[0], 'data_lista']
    
    for x in resto_query.index.tolist():
        imagen_iteracion = resto_query.loc[x, 'data_lista']
        comparacion = similitud_metrica_pixel_to_pixel(imagen_query, imagen_iteracion)
        resultados.append((int(caso_query['indice_main'].iloc[0]), x, 'data_lista', comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                #for campo in caso_query.columns[4:-1].tolist():
                    # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_imagenes, caso_query, resto_query))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

12it [00:00, 185.65it/s]


0 2
1 176
2 304
3 317
4 353
5 441
6 459
7 587
8 658
9 789
10 822
11 915


9it [00:00, 158.43it/s]


0 5
1 198
2 222
3 263
4 496
5 668
6 953
7 1083
8 1186


...

0 285
1 319
2 409
3 464
4 542
5 620
6 647
7 706
8 710
9 721
10 766
11 804
12 920
13 926


16it [00:00, 196.22it/s]


14 1092
15 1213


8it [00:00, 204.22it/s]

0 378
1 404
2 451
3 465
4 806
5 963
6 1179
7 1211


In [44]:
df_resultado_objeto_pixel_to_pixel_filtro_2=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_objeto_pixel_to_pixel_filtro_2.to_csv('df_resultado_objeto_pixel_to_pixel_filtro_2_v2_3.csv', index=False) # modificar


# Caso paisaje

In [45]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_paisajes)):
    
    subdataset_para_comparativa_filtro_2.append(dataset_paisaje_respuestas_completo_bytes_reparado[(dataset_paisaje_respuestas_completo_bytes_reparado['etiqueta']==unicos_paisajes['etiqueta'][i]) &
                                   (dataset_paisaje_respuestas_completo_bytes_reparado['singularidad']==unicos_paisajes['singularidad'][i])])

In [46]:
# empezamos a calcular las metricas
def comparar_imagenes(caso_query, resto_query):
    resultados = []
    imagen_query = caso_query.loc[caso_query.index.tolist()[0], 'data_lista']
    
    for x in resto_query.index.tolist():
        imagen_iteracion = resto_query.loc[x, 'data_lista']
        comparacion = similitud_metrica_pixel_to_pixel(imagen_query, imagen_iteracion)
        resultados.append((int(caso_query['indice_main'].iloc[0]), x, 'data_lista', comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                #for campo in caso_query.columns[4:-1].tolist():
                    # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_imagenes, caso_query, resto_query))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

8it [00:00, 76.48it/s]

0 12
1 164
2 274
3 314
4 318
5 324
6 362
7 466
8 472
9 625
10 871
11 1029
12 1136
13 1143
14 1296
15 1402


24it [00:00, 76.90it/s]

16 1660
17 1854
18 1913
19 1919
20 2147
21 2542
22 2544
23 2753
24 3040
25 3221
26 3314
27 3340
28 3438
29 3486
30 3507
31 3522


35it [00:00, 72.20it/s]


...

18 1178
19 1197
20 1231
21 1295
22 1450
23 1577
24 1888
25 2242


26it [00:00, 78.50it/s]

26 2430
27 2431
28 2625
29 2642
30 2701
31 2755
32 2883
33 3126


37it [00:00, 78.44it/s]

34 3292
35 3390
36 3510


In [47]:
df_resultado_paisaje_pixel_to_pixel_filtro_2=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_paisaje_pixel_to_pixel_filtro_2.to_csv('df_resultado_paisaje_pixel_to_pixel_filtro_2_v2_3.csv', index=False) # modificar
